In [41]:
import numpy as np
import nltk
from nltk import bigrams
import itertools
import pandas as pd


In [43]:
#input data
data = pd.read_csv('csv files/times_df.csv')
data.head()

,Unnamed: 0,world,cup,england,qatar,player,team,southgate,said,game,football,time,match,one,tournament,first,final,league,last,year,would,club,fan,group,wale,fifa,united,two,country,squad,gareth,right,goal,stadium,week,back,three,manager,day,france,season,could,month,play,side,nation,also,million,like,city,...,reopen,perch,tyre,specially,emerges,perfume,renovated,interlocking,glassblower,glare,flame,lonely,installation,wotsit,bauble,random,battleface,earthquake,whatsoever,frustrating,irrational,steppe,welter,simmonds,massacre,alaa,enormity,ambiguous,casson,srinagar,kalash,heed,sousse,nepalese,imprisoned,kyte,unwarranted,territory,indolent,exists,unjustified,kashmir,rod,bealby,clare,sloppy,incompetent,thereby,undermined,invalidated
0,0,0.028616,0.029158,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.050656,0.000000,0.000000,0.000000,0.050576,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.065011,0.000000,0.000000,0.136382,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.068372,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.000000,0.000000,0.000000,0.043025,0.058949,0.0,0.000000,0.0,0.000000,0.000000,0.059334,0.260325,0.000000,0.000000,0.062316,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.144408,0.244331,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.072366,0.074043,0.07547,0.0,0.07584,0.00000,0.0,0.0,0.000000,0.0,0.078591,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.000000,0.000000,0.083572,0.000000,0.000000,0.0,0.060968,0.0,0.000000,0.000000,0.049764,0.000000,0.000000,0.050174,0.000000,0.107174,0.0,0.0,0.000000,0.111047,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.058869,0.000000,0.058253,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.000000,0.0,0.065915,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.065616,0.066859,0.148687,0.000000,0.058642,0.0,0.072314,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.152013,0.069094,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.076972,0.0,0.000000,0.078389,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.125267,0.095729,0.000000,0.122566,0.000000,0.0,0.000000,0.0,0.000000,0.113426,0.000000,0.000000,0.000000,0.056807,0.000000,0.000000,0.0,0.0,0.058359,0.062863,0.0,0.000000,0.000000,0.000000,0.149255,0.06982,0.0,0.066650,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.00000,0.07917,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
def generate_co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    vocab_index = {word: i for i, word in enumerate(vocab)}

    # Create bigrams from all words in corpus
    bi_grams = list(bigrams(corpus))

    # Frequency distribution of bigrams ((word1, word2), num_occurrences)
    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    # Initialise co-occurrence matrix
    # co_occurrence_matrix[current][previous]
    co_occurrence_matrix = np.zeros((len(vocab), len(vocab)))

    # Loop through the bigrams taking the current and previous word,
    # and the number of occurrences of the bigram.
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]
        pos_current = vocab_index[current]
        pos_previous = vocab_index[previous]
        co_occurrence_matrix[pos_current][pos_previous] = count
    co_occurrence_matrix = np.matrix(co_occurrence_matrix)

    # return the matrix and the index
    return co_occurrence_matrix, vocab_index


text_data = [["world", "cup", "england", "qatar", "player", "team", "southgate", "said", "game", "football", "time", "match", "one", "tournament", "first", "final", "league", "last", "year", "would", "club", "fan", "group", "wale", "fifa", "united", "two", "country", "squad", "gareth", "right", "goal", "stadium", "week", "back", "three", "manager", "day", "france", "season", "could", "month", "play", "side", "nation", "also", "million", "like", "city", "win", "place", "four", "manchester", "injury", "since", "say", "iran", "night", "head", "minute", "take", "forward", "second", "made", "make", "even", "new", "champion", "international", "played", "argentina", "ukraine", "coach", "next", "home", "premier", "penalty", "doha", "host", "germany", "messi", "sport", "left", "training", "way", "european", "best", "get", "still", "come", "many", "face", "kane", "end", "well", "chelsea", "november", "state", "people", "past", "start", "scotland", "around", "opening", "scored", "former", "playing", "yet", "national", "set", "may", "captain", "defeat", "victory", "arsenal", "go", "part", "summer", "stage", "supporter", "al", "worker", "draw", "six", "human", "every", "fa", "given", "rashford", "going", "five", "away", "already", "need", "qatari", "ago", "want", "family", "return", "half", "top", "eight", "despite", "saturday", "referee", "put", "bale", "however", "ball", "ten", "campaign", "expected", "another", "real", "including", "much", "deal", "look", "long", "woman", "pitch", "saudi", "sunday", "came", "euro", "feel", "midfielder", "decision", "see", "defender", "point", "russia", "harry", "later", "know", "liverpool", "took", "moment", "migrant", "holland", "james", "good", "career", "brazil", "tottenham", "big", "senegal", "yesterday", "move", "record", "men", "medium", "celtic", "tuesday", "though", "championship", "italy", "hope", "fixture", "kick", "thing", "always", "official", "think", "history", "different", "never", "president", "far", "used", "centre", "round", "spain", "whether", "ronaldo", "lionel", "seven", "without", "today", "question", "police", "run", "might", "chance", "van", "morocco", "portugal", "hour", "due", "man", "footballer", "event", "confirmed", "striker", "little", "english", "told", "work", "something", "name", "become", "show", "performance", "making", "taking", "knee", "among", "mbappé", "north", "madrid", "west", "great", "mean", "friday", "competition", "seen", "beckham", "wednesday", "ticket", "thursday", "result", "body", "missed", "croatia", "instead", "call", "arabia", "hotel", "appearance", "member", "got", "sterling", "video", "meeting", "revealed", "life", "lost", "behind", "per", "australia", "shirt", "support", "statement", "concern", "le", "winner", "level", "south", "usa", "old", "asked", "infantino", "hotspur", "maddison", "thought", "action", "lot", "japan", "major", "early", "goalkeeper", "line", "star", "form", "ever", "biggest", "plan", "perhaps", "almost", "across", "issue", "bellingham", "started", "monday", "belgium", "miss", "talk", "evening", "cent", "involved", "least", "british", "friendly", "stand", "december", "wembley", "middle", "david", "love", "went", "number", "recent", "following", "front", "london", "third", "beat", "social", "african", "saka", "claim", "suffered", "watch", "paris", "trophy", "offer", "better", "beer", "winning", "rather", "park", "help", "whose", "contract", "experience", "bbc", "criticism", "course", "full", "armband", "job", "wearing", "gulf", "everyone", "leave", "taken", "june", "change", "probably", "camp", "rule", "returned", "really", "white", "previous", "marcus", "sporting", "leading", "dream", "led", "news", "called", "begin", "likely", "case", "bit", "young", "sent", "saying", "keep", "ecuador", "american", "september", "break", "doubt", "able", "gone", "fact", "twice", "surgery", "protest", "newcastle", "find", "son", "done", "close", "page", "ground", "give", "although", "desert", "weekend", "fit", "challenge", "claimed", "travel", "gaal", "free", "held", "red", "word", "stay", "hand", "earlier", "room", "followed", "attempt", "barcelona", "quite", "company", "lead", "gay", "became", "itv", "death", "getting", "must", "local", "denmark", "power", "coming", "thousand", "reason", "title", "uefa", "east", "message", "executive", "soccer", "chief", "hard", "governing", "walker", "injured", "federation", "bayt", "prince", "position", "foden", "ready", "working", "association", "flight", "feeling", "fear", "late", "report", "signing", "st", "reach", "attack", "success", "opponent", "found", "business", "qualifying", "role", "winter", "billion", "turned", "lion", "period", "charge", "use", "tv", "focus", "rugby", "party", "received", "series", "story", "sign", "tonight", "small", "often", "understood", "scoring", "crowd", "security", "looking", "welsh", "post", "nothing", "according", "future", "public", "sense", "spent", "scottish", "tie", "bid", "senior", "greatest", "known", "enough", "session", "wanted", "died", "sky", "shot", "open", "paid", "jude", "talent", "ghana", "difficult", "law", "several", "ended", "hit", "looked", "invasion", "ranger", "missing", "office", "arrived", "friend", "morning", "serious", "within", "abuse", "cap", "high", "significant", "named", "towards", "surprise", "exit", "area", "described", "assistant", "authority", "march", "score", "included", "french", "town", "starting", "government", "spot", "clash", "saw", "television", "global", "qualify", "cristiano", "card", "ahead", "flag", "joined", "mind", "watched", "lineker", "felt", "reported", "gary", "outside", "italian", "opportunity", "finished", "everything", "live", "kylian", "feature", "believe", "russian", "sure", "added", "inside", "watching", "progress", "others", "uk", "continue", "committee", "agreed", "eye", "important", "source", "certainly", "available", "announced", "clear", "test", "hoping", "brighton", "pickford", "canada", "couple", "wife", "phillips", "happy", "belief", "transfer", "building", "domestic", "corruption", "midfield", "favourite", "journey", "single", "substitute", "latest", "brought", "homosexuality", "fly", "display", "shown", "idea", "dubai", "matter", "deschamps", "positive", "loss", "cardiff", "force", "allowed", "tell", "strong", "anything", "let", "service", "decade", "decided", "union", "celebration", "leader", "war", "allow", "de", "interview", "jordan", "german", "loan", "gianni", "construction", "bad", "bayern", "bukayo", "everton", "successful", "foot", "conference", "signed", "journalist", "similar", "admitted", "jack", "criticised", "bin", "quality", "certain", "failed", "born", "energy", "began", "sale", "ban", "heart", "wait", "remain", "organiser", "knockout", "vote", "kyle", "spoke", "remarkable", "poland", "milan", "arrest", "alcohol", "growing", "age", "money", "super", "conte", "term", "tomorrow", "mile", "regime", "huge", "beaten", "october", "lack", "meet", "iranian", "contact", "along", "warned", "turn", "rainbow", "lusail", "option", "george", "running", "potter", "coverage", "kept", "arab", "pressure", "debut", "bar", "guardiola", "problem", "meant", "avoid", "opened", "july", "situation", "winger", "travelling", "appeared", "ben", "hold", "switzerland", "remains", "forest", "staff", "whole", "potential", "sir", "kind", "tear", "arrested", "shoulder", "stopped", "showed", "pay", "korea", "henderson", "especially", "selection", "ham", "antonio", "raheem", "afternoon", "nearly", "trying", "louis", "arteta", "answer", "system", "managed", "grealish", "prospect", "rice", "father", "facing", "visit", "var", "either", "provided", "maguire", "john", "window", "child", "written", "twitter", "possible", "mark", "khalifa", "nine", "varane", "moroccan", "someone", "villa", "insisted", "access", "stoppage", "alongside", "confidence", "gave", "gesture", "together", "treatment", "incident", "hungary", "sound", "view", "finally", "stop", "impact", "base", "board", "africa", "professional", "price", "reached", "mother", "general", "bring", "exactly", "compared", "needed", "king", "emirate", "press", "europa", "opposition", "delivered", "control", "rest", "detail", "boy", "europe", "selected", "welcome", "interest", "phil", "officer", "house", "ensure", "battle", "pogba", "street", "reece", "step", "condition", "box", "plenty", "speaking", "leicester", "golf", "macedonia", "fitness", "partner", "reaching", "helped", "absence", "faced", "outcome", "join", "main", "pick", "cut", "trippier", "university", "trip", "recently", "average", "brentford", "extra", "firm", "beyond", "soon", "emerged", "bet", "occasion", "broadcast", "wakrah", "headline", "eve", "derby", "none", "save", "famous", "prove", "mexico", "stood", "angeles", "los", "foreign", "rica", "costa", "leg", "memory", "controversial", "dramatic", "golden", "figure", "yellow", "responded", "nobody", "qualified", "holding", "scorer", "happened", "replaced", "worked", "brother", "leaving", "announcement", "ireland", "increased", "sometimes", "illegal", "footballing", "paul", "glory", "hag", "fifth", "losing", "built", "usual", "thanks", "list", "bennett", "community", "drawn", "suggested", "difference", "sell", "ability", "piece", "brilliant", "subject", "recorded", "individual", "worst", "gabriel", "pep", "rise", "martínez", "prepare", "audience", "amid", "image", "didier", "talking", "effort", "hamstring", "graham", "usually", "xi", "target", "awarded", "considered", "january", "political", "agent", "moved", "meaning", "asian", "recall", "offside", "hampden", "wilson", "coast", "juventus", "suddenly", "female", "wrong", "ferguson", "munich", "mood", "ranked", "newspaper", "dutch", "fine", "defence", "screen", "cost", "channel", "wear", "key", "read", "band", "standing", "tough", "palace", "actually", "yes", "shock", "vast", "air", "sun", "total", "tried", "williams", "whistle", "race", "previously", "aware", "eu", "mac", "knew", "potentially", "carabao", "near", "fund", "greater", "rob", "living", "albion", "lgbt", "train", "maradona", "easy", "beating", "buy", "tent", "ultimate", "wonder", "serbia", "china", "assist", "wanting", "boot", "noise", "earn", "present", "worth", "understands", "diego", "cause", "prepared", "fashion", "book", "maybe", "linked", "corner", "marc", "wall", "conversation", "discrimination", "keen", "presence", "restaurant", "enjoying", "beach", "special", "fully", "disappointment", "struck", "comment", "possibility", "scan", "technology", "hospital", "provide", "steve", "behaviour", "share", "longer", "seem", "technical", "premiership", "released", "relationship", "giant", "traditional", "la", "knocked", "hove", "space", "net", "forced", "bowen", "arguably", "offered", "scene", "fell", "britain", "heat", "wolf", "allister", "photo", "bench", "schedule", "throughout", "denied", "sustained", "changing", "short", "lose", "visitor", "austria", "western", "quarter", "black", "ambassador", "stake", "atmosphere", "road", "postecoglou", "accommodation", "negative", "rain", "killed", "considering", "based", "jesus", "sitting", "central", "colour", "except", "planned", "venue", "push", "contest", "happen", "rival", "light", "met", "immediately", "visiting", "netherlands", "placed", "agreement", "scandal", "ac", "heading", "row", "website", "simply", "youth", "stone", "attention", "order", "hear", "confident", "fresh", "organisation", "choice", "brazilian", "instagram", "financial", "benefit", "large", "definitely", "overcome", "preparation", "encounter", "backed", "speak", "majority", "green", "refused", "knowledge", "aim", "yard", "treated", "amount", "generation", "caused", "try", "kicked", "market", "fc", "true", "holiday", "spur", "style", "seemed", "scheduled", "attend", "fulham", "shortly", "poor", "limited", "rooney", "toney", "labour", "boxing", "whatever", "skill", "council", "wore", "somehow", "juranovic", "wayne", "ruled", "field", "uruguay", "seat", "racist", "competing", "pelé", "jones", "dropped", "joining", "fourth", "imagine", "allegation", "klopp", "heard", "complete", "anthony", "table", "discussion", "evidence", "hundred", "proud", "include", "build", "enjoy", "neville", "accused", "republic", "fight", "temperature", "nottingham", "award", "dispute", "broadcaster", "plus", "proved", "moving", "narrative", "ambition", "personal", "kid", "walk", "hurt", "academy", "america", "sat", "reality", "risk", "changed", "remember", "bournemouth", "enrique", "court", "delay", "seems", "carsley", "jürgen", "defending", "surrey", "understand", "struggled", "kalvin", "spanish", "capital", "bus", "facility", "land", "operation", "arm", "erik", "luxury", "daily", "initially", "importance", "zone", "neymar", "argentinian", "picked", "setback", "notice", "secure", "chilwell", "aston", "spell", "sort", "emotion", "fellow", "suffering", "arabian", "eriksen", "mané", "dijk", "richarlison", "christmas", "kaili", "advertising", "trafford", "culture", "cross", "section", "turning", "straight", "ceremony", "glasgow", "ivory", "investigation", "celebrating", "familiar", "safe", "responsibility", "prior", "defensive", "tite", "crucial", "parliament", "policy", "block", "entire", "reputation", "conceded", "knowing", "february", "lie", "gakpo", "combined", "continued", "opener", "particularly", "consecutive", "date", "phone", "threat", "rejected", "allowing", "dortmund", "southampton", "celebrated", "filled", "dier", "school", "royal", "follow", "deep", "goalless", "region", "determined", "raised", "queiroz", "recovery", "regularly", "supposed", "lunchtime", "surely", "touch", "frappart", "becoming", "appeal", "posted", "ranking", "low", "note", "celebrate", "hen", "martin", "criminal", "act", "aircraft", "fall", "berhalter", "size", "michael", "controversy", "pundit", "else", "borussia", "hardly", "joke", "viewer", "represented", "seeing", "tour", "pace", "ask", "unprecedented", "drive", "walked", "august", "eddie", "expecting", "beginning", "brand", "account", "undergone", "backing", "particular", "returning", "hosting", "owner", "turkey", "sold", "anthem", "asking", "bringing", "link", "served", "arrive", "irish", "project", "circumstance", "ramsdale", "header", "trent", "fee", "cheer", "clock", "struggling", "equaliser", "proposal", "thinking", "pair", "airway", "effect", "remained", "trial", "setting", "dominated", "packed", "suspect", "dad", "alone", "highest", "showing", "countdown", "booed", "revenue", "ifab", "measure", "created", "amnesty", "sydney", "quickly", "cancer", "announce", "sadio", "handed", "werner", "advice", "strange", "giving", "closer", "private", "minister", "declan", "picture", "remove", "travelled", "regard", "empty", "politics", "threatened", "concerned", "removed", "caught", "keane", "debate", "value", "extended", "joy", "spectacular", "urc", "chairman", "easily", "represents", "expect", "wahl", "william", "significantly", "carefully", "send", "tunisia", "arrival", "carlos", "longest", "howe", "learn", "attacking", "increasingly", "inspired", "dublin", "commercial", "lived", "discus", "plane", "artist", "powerful", "legacy", "wake", "prompted", "focused", "respect", "approach", "dismissed", "qualification", "secret", "add", "speed", "putin", "completed", "depth", "blatter", "robert", "brussels", "stick", "apparent", "santos", "gordon", "drew", "converted", "souq", "leaf", "seeking", "promise", "worse", "class", "disappointing", "door", "carry", "manage", "welcoming", "camera", "response", "using", "dark", "custody", "versus", "cameroon", "upset", "requested", "damage", "unlikely", "clean", "covid", "alex", "elite", "prime", "visited", "ordered", "ziyech", "airline", "hosted", "cultural", "request", "lad", "consider", "identity", "sing", "care", "molineux", "hernández", "impression", "callum", "owned", "securing", "parent", "department", "anticipated", "ukrainian", "wanderer", "continues", "ending", "emotional", "olympic", "promised", "betting", "credit", "secretary", "egypt", "surprised", "modric", "drama", "continent", "taylor", "recovered", "hat", "waiting", "flick", "performed", "excitement", "influence", "banned", "enjoyed", "celebrity", "attitude", "male", "excellent", "expects", "stamford", "bridge", "perfect", "ran", "exhibition", "staged", "refereeing", "indeed", "upon", "pointed", "luis", "wag", "process", "impressive", "kompany", "stunning", "blue", "truth", "earned", "eric", "putting", "gallagher", "wide", "rewarded", "passport", "appearing", "tchouaméni", "publicly", "rate", "forget", "relaxed", "pleased", "invitation", "mikel", "create", "pushed", "qualifier", "deliver", "beautiful", "wage", "hero", "improved", "sentence", "achievement", "thigh", "address", "departure", "soft", "regular", "intense", "illness", "aged", "duty", "painful", "highlight", "effectively", "joe", "truly", "pot", "affected", "newsletter", "lengthy", "tourist", "exchange", "doctor", "prison", "desire", "hell", "relief", "bidding", "nice", "camel", "honour", "leeds", "era", "planet", "peru", "experienced", "study", "frank", "drone", "saved", "lawyer", "development", "racism", "struggle", "thuram", "rare", "shape", "flying", "vincent", "demand", "unbeaten", "incredibly", "mep", "clarke", "cash", "symbol", "phase", "appointed", "crystal", "deeply", "historically", "double", "cry", "quietly", "increase", "extent", "thrown", "husband", "information", "hazard", "scrutiny", "featured", "defeating", "hub", "warning", "confirm", "mostly", "resolution", "church", "example", "alexis", "disappointed", "site", "listed", "roberto", "judge", "sacked", "shared", "benfica", "participation", "moscow", "lampard", "column", "drink", "thrilling", "booking", "ange", "alleged", "historic", "pause", "excited", "interested", "lift", "impressed", "koulibaly", "endured", "mp", "bold", "relation", "unable", "expert", "strike", "physical", "produced", "mentally", "furuhashi", "violence", "tiny", "spending", "status", "hennessey", "prize", "pride", "brings", "pulled", "direct", "raphaël", "improving", "allegedly", "specialist", "regragui", "nature", "appears", "aside", "drop", "planning", "hassan", "johnston", "deny", "gold", "deserved", "superstar", "klinsmann", "sympathy", "recover", "dashed", "medical", "miami", "cole", "additional", "involvement", "busy", "everybody", "character", "represent", "australian", "education", "taker", "pain", "boost", "hole", "oil", "dressed", "sweden", "height", "ivan", "mouth", "estimated", "bos", "hakimi", "simple", "sidelined", "jadon", "sancho", "distance", "punishable", "enduring", "burnley", "prayer", "gas", "decisive", "childhood", "questioned", "granted", "via", "asia", "tourism", "thus", "attracted", "failing", "convince", "feared", "vowed", "happens", "explained", "choose", "border", "ajax", "weight", "lifted", "fair", "song", "keeping", "trust", "kanté", "holder", "complaint", "task", "slot", "muscle", "attacker", "seed", "cast", "infrastructure", "stride", "colak", "director", "islamic", "committed", "spread", "mental", "suit", "mentality", "kicking", "bedouin", "sang", "proceeding", "neither", "art", "psg", "presenter", "calm", "trouble", "coaching", "actor", "refusing", "ruling", "economy", "broke", "management", "mosque", "aspect", "hearing", "protester", "permanent", "motivation", "occurred", "understanding", "norway", "paying", "heavy", "crazy", "girlfriend", "honest", "andy", "aaron", "midday", "dundee", "published", "health", "covering", "davy", "loved", "path", "lucky", "eva", "mason", "preparing", "review", "shop", "intelligence", "deadline", "ml", "suggesting", "abraham", "sponsorship", "seemingly", "anyone", "closed", "dressing", "triumph", "canadian", "standard", "accurate", "shabka", "expectation", "mansion", "exercise", "declared", "porto", "colleague", "delayed", "bought", "fernández", "mohamed", "seized", "expressed", "halfway", "girl", "completely", "cody", "welcomed", "sheffield", "silva", "laughing", "olympics", "agree", "clearly", "urged", "modern", "chris", "film", "officially", "craft", "landed", "swept", "hamad", "unit", "bank", "conservative", "finalist", "bag", "range", "eze", "lifetime", "thomas", "delivery", "groin", "ali", "clothing", "earth", "strict", "jersey", "downturn", "investment", "split", "ngland", "catch", "minor", "postponed", "reflect", "aimed", "musiala", "decide", "resume", "guéhi", "superior", "lloris", "eliminated", "document", "greek", "april", "frustration", "imagined", "partly", "temporary", "unlike", "discussed", "widely", "possibly", "bribery", "subsequent", "entirely", "unexpected", "vladimir", "photograph", "budweiser", "heavily", "defeated", "retirement", "convinced", "shakhtar", "suspended", "compete", "hernandez", "omitted", "deciding", "reflected", "dead", "capacity", "practice", "supporting", "achilles", "aggressive", "arriving", "realised", "bay", "county", "retire", "amazing", "complicated", "hopeful", "protect", "princess", "hot", "kalidou", "arabic", "programme", "experiencing", "recovering", "achieved", "levitt", "addition", "citizen", "voice", "rich", "apart", "bear", "victoria", "wished", "ramsey", "daughter", "network", "slightly", "happening", "incredible", "smaller", "slipped", "capped", "wider", "conduct", "required", "chinese", "detained", "bergwijn", "mirren", "cricket", "tale", "hoped", "introduced", "griezmann", "oliver", "strongly", "explain", "peak", "misery", "rivalry", "initial", "strength", "charlton", "beneath", "sit", "fun", "autumn", "wenger", "participating", "lewandowski", "bright", "requires", "competitive", "employer", "safety", "raucous", "natural", "critical", "spend", "observer", "headed", "booked", "josip", "smile", "pas", "advantage", "tweeted", "sobering", "population", "dry", "turf", "insists", "boycott", "argument", "staging", "forthcoming", "kenny", "documentary", "extend", "reduce", "neighbour", "ignoring", "covered", "featuring", "compelling", "aire", "buenos", "successor", "backdrop", "wiegman", "representative", "repeat", "senegalese", "track", "löw", "calendar", "determination", "approached", "involving", "retired", "homophobic", "spoken", "iconic", "industry", "ease", "lucrative", "czech", "inevitably", "online", "sight", "hanging", "bottom", "advised", "anywhere", "singing", "replacement", "item", "delighted", "nickname", "discovered", "anyway", "stock", "finish", "stayed", "festival", "chef", "szczesny", "kosovo", "medal", "needing", "refugee", "odds", "seek", "ow", "scot", "politician", "suite", "severe", "hair", "endorsement", "complex", "widespread", "passing", "birmingham", "conor", "fewer", "targeted", "tuned", "knock", "portuguese", "steven", "grab", "necessary", "broken", "anfield", "gathered", "smith", "epic", "pen", "ahmad", "legal", "colombia", "exciting", "dozen", "freedom", "explains", "robbie", "chart", "spaniard", "moral", "praise", "fracture", "lewis", "christian", "drinking", "opted", "barisic", "routine", "chosen", "rank", "common", "drunk", "homeland", "organised", "thani", "departed", "condemned", "reading", "sixth", "golo", "passenger", "background", "chapter", "includes", "raid", "respectively", "pound", "attract", "artificial", "aurélien", "speech", "truck", "entertainment", "hypocrisy", "verdict", "explanation", "trapped", "grown", "improvement", "surface", "stepped", "data", "cover", "advanced", "supreme", "zurich", "prosecutor", "hospitality", "established", "psv", "athletic", "various", "trump", "defended", "testing", "hudson", "rarely", "pragmatic", "ship", "mention", "rocket", "water", "cruel", "advancing", "carrying", "possession", "tag", "grew", "largest", "sudden", "pole", "pint", "resident", "restriction", "bruno", "consistency", "bribe", "marked", "product", "doo", "thank", "technique", "fastest", "hug", "grant", "toone", "search", "handle", "century", "stopping", "fill", "episode", "peter", "escape", "charged", "designed", "presented", "zelensky", "castillo", "specific", "original", "displayed", "shootout", "victim", "concede", "providing", "carried", "trajectory", "largely", "storm", "ignored", "maternity", "reminded", "reporter", "footage", "boo", "collision", "japanese", "concert", "suggestion", "podcast", "release", "aberdeen", "supply", "robertson", "cold", "salt", "remind", "expression", "foul", "breach", "birth", "stance", "paper", "deserve", "campaigner", "glendinning", "guide", "strengthen", "brief", "fernando", "kingdom", "volley", "lower", "indicated", "leipzig", "barrage", "arnold", "cheshmi", "engineer", "reader", "encourage", "hailed", "jail", "desperately", "jassim", "guard", "lee", "prefer", "receive", "depend", "sum", "allegiance", "queen", "duo", "manuel", "replace", "guarantee", "sparked", "listen", "difficulty", "highly", "singer", "alternative", "ankle", "theo", "guaranteed", "urge", "musk", "invested", "hill", "fate", "adidas", "dancing", "hjulmand", "stated", "debt", "punter", "grasp", "omission", "person", "rehabilitation", "differently", "recalled", "neil", "hopefully", "dinner", "hugo", "agency", "consequence", "fired", "sex", "pandemic", "donetsk", "edge", "moore", "sending", "tower", "organise", "faith", "cancelled", "mistake", "increasing", "affect", "wealth", "computer", "contested", "accusation", "adopt", "ampadu", "formal", "grass", "begun", "contingent", "hayes", "yoshida", "toughest", "greeted", "approval", "enthusiasm", "seriously", "musah", "gained", "settled", "extremely", "ligament", "racial", "blame", "provisional", "efl", "sussex", "factor", "perform", "andrew", "letter", "complained", "closest", "sideline", "expanded", "resulted", "challenging", "striking", "commit", "salman", "robson", "spectator", "predicted", "goalscorer", "floodlight", "successfully", "kieran", "filmed", "afterwards", "obviously", "martinelli", "jim", "route", "dreaming", "argentine", "onto", "agreeing", "postponement", "writer", "describe", "gang", "drum", "roar", "initiative", "athlete", "insight", "offering", "extraordinary", "imminent", "falling", "closing", "midway", "pool", "kevin", "cantona", "guest", "cheering", "contention", "attended", "crisis", "weakness", "curse", "ipswich", "fail", "kiwior", "maximum", "sponsor", "believed", "applause", "resort", "physically", "worry", "unknown", "element", "compare", "cope", "owen", "learning", "nightmare", "plight", "adopted", "vaccine", "nasser", "priority", "infamous", "frenchman", "hugely", "genuine", "visa", "audition", "mini", "heckled", "improve", "similarly", "shouting", "summit", "atletico", "limped", "noticed", "popular", "poll", "bow", "johnstone", "bizarre", "kyiv", "california", "equipment", "highlighted", "student", "museum", "rose", "availability", "punch", "sam", "arena", "passed", "personally", "barely", "admits", "genius", "pr", "conclusion", "undergoing", "salah", "concluded", "belgian", "chiellini", "contrast", "calling", "awkward", "mobile", "approve", "giggs", "compensation", "language", "intention", "yorkshire", "thawadi", "surroundings", "immense", "motivational", "sepp", "tunnel", "telling", "favoured", "pub", "waving", "middlesbrough", "broad", "formula", "exposed", "veteran", "topic", "finding", "pull", "secured", "excuse", "alistair", "wrote", "eventually", "swansea", "kulusevski", "subjected", "virgil", "fter", "dating", "fantastic", "aerial", "pass", "managing", "participant", "otherwise", "gerrard", "tactic", "fulfilling", "whatsapp", "warner", "gap", "jamie", "briefly", "ayew", "burton", "rejection", "adding", "walid", "relatively", "commitment", "platini", "metropolitan", "performing", "inspiration", "succession", "directive", "arsène", "prevented", "collapsed", "houston", "outstanding", "entering", "strain", "sheikh", "earnings", "fire", "analysis", "expense", "greece", "demonstration", "relegated", "revealing", "finger", "hakim", "showpiece", "lovely", "bronckhorst", "offence", "guy", "wakra", "copa", "cutting", "coup", "spokesman", "coveted", "fortune", "noted", "duggan", "lawmaker", "wrongdoing", "ronald", "koeman", "cabin", "plot", "reporting", "representing", "apple", "learnt", "stint", "wing", "webb", "failure", "accepted", "supported", "environmental", "tension", "beloved", "army", "pedro", "tactical", "declined", "abdulaziz", "military", "benzema", "fox", "direction", "placing", "roy", "spice", "automatically", "app", "sanction", "bill", "rid", "eto", "foster", "appear", "riding", "muslim", "garden", "flew", "accidental", "higher", "vinícius", "albania", "entry", "carnival", "gambling", "ultimately", "decent", "ruler", "nurse", "firmly", "car", "justice", "switch", "shipped", "banner", "candidate", "paint", "twin", "tiger", "distant", "palmeiras", "finest", "monitoring", "model", "autocratic", "talented", "renowned", "tehran", "trained", "underwent", "advert", "concrete", "suspicion", "promotion", "bed", "deemed", "tax", "transformed", "pier", "suggests", "ministry", "unhappy", "brutal", "sigh", "pulling", "assured", "broadcasting", "reportedly", "cleverly", "receiving", "length", "expensive", "inter", "rumour", "overall", "solo", "blowing", "explore", "negotiation", "blamed", "lens", "sociedad", "embraced", "apartment", "crossed", "virus", "closely", "marriage", "fury", "breaking", "native", "associated", "macron", "eduardo", "camavinga", "fast", "nigeria", "contains", "terrific", "angry", "enter", "god", "lopetegui", "julen", "missile", "eindhoven", "sea", "assessed", "invited", "edition", "nh", "ratcliffe", "cool", "gift", "remembered", "par", "shake", "flown", "intent", "remaining", "critic", "founder", "write", "outlet", "dr", "hang", "chile", "stable", "bell", "israeli", "bae", "application", "sanctioned", "favour", "newport", "overnight", "spotted", "elsewhere", "aubameyang", "professor", "landscape", "ass", "rb", "memorable", "mr", "lucas", "judged", "dazzling", "stepping", "detention", "prevent", "restore", "wolverhampton", "launched", "chaotic", "costing", "progressing", "mass", "airbus", "ryder", "versatile", "oxford", "launch", "pretty", "stv", "anniversary", "wave", "clarkson", "store", "alert", "jarrod", "rowe", "unique", "picking", "string", "volume", "strategy", "fortunate", "outfit", "shame", "weapon", "follower", "attacked", "commentator", "jeremy", "iraq", "reminder", "fernandes", "ecuadorian", "limit", "ne", "danish", "creating", "trade", "zealand", "transition", "wine", "studio", "scheduling", "careful", "overlooked", "bigger", "regret", "dividend", "diyarbakir", "moukoko", "dominance", "solace", "worldwide", "metro", "swung", "sputnik", "expires", "acting", "tradition", "jamal", "sits", "responsible", "baccus", "pope", "engage", "dalic", "johnson", "ferdinand", "reduced", "looming", "airspace", "nick", "nowhere", "sought", "quiet", "production", "credential", "billionaire", "spectacle", "enters", "loud", "strongest", "permission", "experimental", "ridiculous", "winterbottom", "sánchez", "simulator", "elimination", "ho", "commentary", "keeper", "awaits", "astonishing", "inspiring", "food", "underpinned", "khor", "agonising", "ukfpu", "rouzbeh", "mendy", "timing", "appetite", "alf", "emery", "hibs", "luke", "sweet", "reign", "rape", "saint", "smiling", "fairytale", "sheriff", "superb", "uncapped", "conclude", "soil", "robe", "judging", "hendry", "uncomfortable", "thorny", "likelihood", "resolve", "goalscoring", "scoresheet", "herdman", "moriyasu", "fundamental", "sheet", "perspective", "warm", "washing", "elliott", "requirement", "occasional", "hardest", "starfelt", "thrilled", "environment", "airport", "counter", "skipper", "kill", "labourer", "bat", "balance", "aspire", "oh", "heritage", "flat", "teenager", "playmaker", "regarded", "consideration", "meantime", "triumphant", "revival", "unless", "cheek", "banker", "dugout", "suggest", "banana", "crashed", "coutinho", "engaged", "investigated", "presumably", "rayyan", "dangerous", "endeavour", "impossible", "cardiac", "exhausting", "swathe", "embarked", "etihad", "destiny", "admit", "lacked", "tea", "circle", "jacob", "estimate", "luck", "allan", "contender", "waqif", "unvaccinated", "lisbon", "peace", "noisy", "taste", "cheika", "exception", "gripping", "nervous", "molloson", "exploit", "relish", "inviting", "nou", "deeper", "hitting", "narrow", "casting", "require", "beyoncé", "rush", "button", "hometown", "ear", "proper", "independent", "flamini", "lamb", "happily", "obliged", "incentive", "publicity", "minus", "haribo", "india", "promote", "blessing", "hatate", "reassure", "cnn", "clip", "annual", "ancelotti", "flashed", "aguerd", "cloud", "unfair", "duration", "raising", "watford", "swiss", "finishing", "goalscorers", "extreme", "emile", "aboubakar", "undoubtedly", "consistent", "jules", "easier", "payment", "stuffed", "jeopardy", "neutral", "institution", "evident", "onelove", "tannadice", "allocation", "flank", "signalled", "calf", "fake", "currently", "cherished", "withdrew", "sideshow", "wound", "evil", "tango", "occasionally", "eden", "tail", "beauty", "older", "labelled", "sheer", "ministerial", "scoreline", "drug", "glazer", "alcaraz", "boca", "reminding", "challenger", "sunlit", "conditioning", "sprint", "oleksandr", "worrying", "litre", "humiliating", "norwich", "seaside", "screened", "pyramid", "descended", "bobby", "signature", "jota", "integral", "discussing", "diary", "ndiaye", "maturity", "glimpse", "hailing", "angel", "accent", "diet", "attending", "blessed", "laugh", "scrap", "liga", "rotation", "demanded", "investigator", "tweak", "terrorist", "vale", "reflection", "amrabat", "stephen", "passion", "guided", "smashing", "mine", "wary", "normally", "tuchel", "tragedy", "transmission", "identify", "tone", "stereotype", "embarrassing", "mitchell", "stress", "jeered", "blasted", "dusk", "minamino", "dakar", "generous", "policing", "agenda", "daizen", "maeda", "distinguished", "miguel", "boyle", "flowed", "squeeze", "announcing", "ramos", "afford", "serve", "withdrawal", "delight", "therefore", "adventure", "coached", "resigning", "phrase", "basis", "obvious", "sarina", "nationality", "coffee", "observation", "delivering", "orsato", "famine", "feast", "appreciate", "eastern", "starkly", "properly", "contemplating", "charles", "surrounded", "stricter", "fairness", "unclear", "shortage", "prominent", "substituted", "complain", "dylan", "latter", "en", "avid", "mohammed", "bein", "withdrawn", "chatting", "fortnight", "faw", "root", "directly", "burst", "ballon", "cox", "aliou", "cissé", "chesney", "allegri", "purpose", "tend", "existed", "antoine", "realistic", "youngster", "souk", "sustainability", "afforded", "coman", "kingsley", "panzeri", "primarily", "raine", "dual", "trainer", "doubled", "bottle", "engine", "edinburgh", "essentially", "fuelled", "plastic", "containing", "hughton", "longstanding", "fallen", "fractious", "floor", "villain", "knocking", "nkunku", "dormitory", "wishing", "swiftly", "immediate", "comfortable", "challenged", "progressed", "eighth", "nistelrooy", "haller", "strictly", "riley", "füllkrug", "jamaica", "tony", "editor", "punishing", "talked", "sydorchuk", "dance", "shield", "constantly", "destined", "substitution", "discover", "successive", "embassy", "author", "unai", "emma", "congress", "music", "spoil", "henry", "opposite", "booze", "dynamo", "masterclass", "corden", "dismissal", "diplomatic", "skyline", "fault", "bonus", "conversion", "equal", "awarding", "prolific", "unexplained", "anxious", "ally", "swing", "glitzy", "influential", "kensington", "grip", "prepares", "patriotic", "error", "mentioned", "wild", "scenario", "korean", "switched", "circulated", "ethan", "freak", "mate", "greatly", "ideally", "trailed", "regional", "beautifully", "acknowledged", "walking", "joachim", "hansi", "endrick", "port", "walter", "ed", "wind", "shooting", "ibrox", "engagement", "huddersfield", "pure", "emergency", "rodrygo", "júnior", "prevailed", "embarrassment", "indie", "fuel", "rüdiger", "delegate", "suárez", "lisandro", "arrogant", "kyogo", "bored", "matchday", "daunting", "laid", "logo", "empire", "alarm", "barring", "everywhere", "sarmiento", "mitoma", "persuaded", "research", "bundesliga", "gregg", "surround", "hojbjerg", "tune", "sustainable", "reaction", "directing", "minimum", "enemy", "filed", "damning", "comeback", "bleu", "rom", "tammy", "hollywood", "ice", "excellence", "kit", "michel", "marseille", "quick", "throw", "inclusion", "undergo", "considerably", "outrage", "variety", "facial", "activist", "inspire", "konaté", "ibrahima", "thomson", "produce", "lawwell", "customer", "hired", "marino", "kharkiv", "surrounding", "expertise", "profile", "werder", "bremen", "morgan", "subsequently", "cable", "design", "developing", "unfortunately", "accelerated", "completing", "rover", "briton", "leapt", "julián", "álvarez", "grainy", "separate", "sheen", "midland", "rapidly", "argue", "sleep", "rebel", "modest", "legitimacy", "pepe", "reward", "estádio", "dragão", "ascot", "achraf", "assembly", "prostate", "sword", "socialist", "galaxy", "captained", "fred", "indictment", "upamecano", "polish", "employed", "amini", "boosted", "associate", "agnelli", "bitter", "silverware", "karim", "deprived", "understudy", "howard", "tap", "leandro", "scream", "trading", "shrug", "develop", "ardent", "interim", "neutrality", "melrose", "communist", "snow", "boast", "ceferin", "platform", "giorgi", "luka", "punishment", "brick", "heaven", "absolute", "famed", "fed", "envelope", "burn", "slump", "pose", "kai", "focusing", "wet", "lifelong", "grassroots", "jorge", "northwest", "threw", "quiz", "corporation", "blazed", "pivotal", "kurdish", "antony", "ad", "maverick", "talisman", "helping", "marketing", "activity", "joint", "enshrined", "hall", "jimmy", "leadership", "searched", "pack", "messaging", "centenary", "ferocious", "steak", "brewdog", "pif", "typically", "violent", "hey", "hijab", "breakthrough", "ivanov", "roof", "gokce", "richie", "rising", "crew", "summanen", "campaigning", "streaming", "gain", "welfare", "abramovich", "arranged", "valencia", "lula", "stéphanie", "sunderland", "ligt", "danger", "carol", "count", "enzo", "convention", "bolsonaro", "collected", "operating", "filming", "touched", "marco", "flynn", "overseas", "gradually", "examination", "imposed", "lockdown", "vital", "housing", "patient", "tree", "treat", "meme", "richard", "diplomat", "clothes", "earning", "pursuit", "baker", "akbar", "rwanda", "shoot", "glass", "behalf", "frankfurt", "buyer", "aké", "entered", "festive", "compound", "attempting", "pregnant", "avoided", "dp", "throwing", "mcallister", "israel", "monarch", "schoolboy", "liam", "accusing", "premium", "directed", "predicts", "sand", "miracle", "kidd", "jodie", "hostel", "seedorf", "sometime", "false", "naked", "supplier", "dragon", "triumphed", "desperate", "supplying", "pereira", "meanwhile", "continuation", "outstay", "underperform", "yarmolenko", "birighitti", "duchess", "invaded", "díaz", "michelle", "gaz", "medieval", "proposed", "registered", "village", "interpol", "breakfast", "naples", "extradition", "claiming", "chant", "oligarch", "corp", "absolutely", "existence", "jet", "corniche", "constant", "tantholdt", "heartbreak", "wotherspoon", "sounded", "devastated", "ned", "voter", "believing", "plugged", "extends", "radio", "draining", "brian", "mexican", "file", "staying", "mix", "rodgers", "coronavirus", "mcgregor", "inquest", "rub", "describes", "approved", "relishing", "growth", "willing", "encouraging", "arbitration", "warsop", "device", "corporate", "uncertainty", "traffic", "burning", "email", "cluster", "intend", "emir", "searching", "abroad", "kitchen", "accommodate", "decline", "exceptional", "conviction", "destination", "sequence", "gutted", "socially", "koundé", "resilience", "hiding", "imposing", "personality", "recruiting", "counteract", "relax", "appropriate", "thin", "inflicting", "feat", "criticising", "nfl", "eriksson", "punish", "spotlight", "wish", "sampaio", "recognisable", "hewitt", "ryan", "hemmingham", "getafe", "fraught", "divided", "kieffer", "shadow", "boring", "tranquillity", "scaloni", "shevchenko", "olivier", "giroud", "redmayne", "rightly", "bewildering", "mccrae", "irrelevance", "lined", "gripped", "whittle", "vibe", "wane", "format", "virtual", "jiménez", "guess", "brace", "andriy", "risked", "allen", "speculation", "spring", "ancient", "significance", "gear", "imperative", "astonished", "cheque", "detailed", "bureau", "dependent", "lodged", "hugged", "harder", "furniture", "orange", "crown", "excluded", "shook", "underlined", "jamboree", "erhahon", "worried", "shaw", "wherever", "falcon", "exclusively", "foreigner", "sharpness", "hurry", "stuff", "ride", "coronation", "scent", "whoever", "mission", "spence", "tripped", "torturous", "mcginn", "showcase", "mclean", "ust", "headquarters", "racing", "wandered", "thoughtful", "lasting", "borna", "blistering", "exhibit", "swirled", "load", "planted", "dynamic", "alive", "insurer", "normal", "pitted", "somewhat", "pinnacle", "illusion", "younger", "hottest", "stamen", "sprain", "plagued", "earliest", "distinction", "enormous", "atrocity", "highway", "numerous", "tenacity", "macedonian", "crammed", "overseen", "calmness", "vehemently", "operator", "drenched", "feedback", "descend", "counted", "learned", "attendance", "thrill", "insult", "pastime", "tamim", "flux", "pristine", "wisdom", "booing", "recording", "brink", "goa", "bird", "socceroos", "ayrshire", "teacher", "heartache", "universal", "gentle", "oldest", "tomori", "fikayo", "workplace", "nicest", "sting", "magazine", "areth", "piqué", "adam", "heartbreaking", "horizon", "reassuring", "logistics", "chess", "shining", "laughable", "adjusting", "shaking", "dalglish", "inform", "exited", "mehdi", "taremi", "vision", "behave", "smarter", "emulate", "grateful", "restart", "slogan", "select", "abject", "catching", "emerge", "diminutive", "nassr", "stretching", "testament", "ou", "tally", "slip", "noticeable", "epilogue", "unnoticed", "thriving", "complacent", "consumption", "enclosed", "humanitarian", "unused", "crack", "succeed", "seizing", "manner", "transgender", "lloyd", "redemptive", "healthy", "turnaround", "please", "homework", "assertion", "stimulate", "bounou", "devastating", "applied", "conspicuous", "recovers", "seeding", "impress", "driving", "magic", "hostile", "tyranny", "depends", "acutely", "spurious", "equally", "wildly", "straightforward", "precaution", "unchanged", "paisley", "rebuilding", "bloody", "performer", "counterpart", "peninsula", "sadly", "banging", "interesting", "watzke", "arthur", "signal", "managerial", "fitted", "proceeded", "blow", "powerfully", "exemplary", "leonardo", "brave", "dawn", "shaped", "compatriot", "rampant", "snapped", "admitting", "somewhere", "reset", "hinted", "rattled", "naturally", "cue", "hayward", "cummings", "dbu", "ioc", "massive", "tightly", "primary", "answered", "renaissance", "explicitly", "blueprint", "waste", "outsider", "movie", "horror", "climax", "zlatko", "pleasure", "touted", "recrimination", "symbolic", "enlightening", "joked", "climate", "contrive", "domestically", "rubbish", "inclusivity", "hibernian", "saleh", "drunken", "rampage", "keanu", "mcternan", "liked", "forecast", "kiss", "analyst", "cake", "uae", "appraisal", "khater", "xhaka", "granit", "upheaval", "type", "thibaut", "distilled", "sustaining", "deliberate", "shaqiri", "samoura", "bullingham", "ugly", "seventh", "cruise", "greenwood", "underdog", "reunion", "yeah", "civil", "rancorous", "architectural", "definition", "letissier", "unwanted", "zero", "unavailable", "relegation", "cruciate", "arrogance", "quest", "halt", "pepi", "honduras", "cameron", "contrary", "blur", "choreographed", "buzz", "thierry", "beside", "frailty", "center", "midweek", "update", "ok", "fence", "wonderful", "laced", "billed", "breath", "aviva", "recognising", "suspension", "landmark", "generate", "effective", "spelling", "carillion", "curious", "merit", "sacrifice", "zagreb", "dinamo", "sweeping", "replied", "southwest", "pursue", "rollercoaster", "resting", "interior", "ignore", "blazing", "elaborate", "tanguy", "grammys", "dreaded", "emphasis", "cheap", "diagnosed", "bunch", "wrapper", "typical", "dan", "developed", "criminalise", "firing", "monologue", "armed", "diligence", "finland", "cheery", "schmidt", "reel", "accomplished", "seated", "york", "assessing", "metre", "anger", "nike", "pakistan", "shaun", "sensational", "handpicked", "burner", "posse", "capable", "sexual", "feeding", "excelling", "exact", "blemish", "emiliano", "intends", "praised", "transpired", "passage", "suffer", "demise", "handshake", "sober", "rock", "compensate", "rubbing", "lyon", "bruyne", "duel", "reform", "stranger", "heaving", "tagliafico", "contemporary", "dress", "lifting", "overwhelmed", "disappeared", "hiatus", "motorway", "heaviest", "rigorous", "smell", "poised", "acceptance", "introduction", "enable", "apologise", "dilemma", "portsmouth", "eased", "collective", "screaming", "dragged", "bose", "rfu", "overload", "croatian", "risen", "us", "fluent", "tape", "map", "threaten", "devoured", "fix", "lame", "urging", "documented", "plaudit", "erling", "haaland", "traditionally", "homophobia", "exhaustion", "brugge", "brendan", "practising", "firmino", "ralston", "insist", "informed", "intriguing", "hunt", "sends", "plush", "servant", "instruction", "stare", "meek", "fraser", "forster", "san", "urgency", "sensation", "identified", "category", "coalition", "exploring", "boat", "mocking", "spezia", "raisi", "miserable", "talksport", "cabinet", "sod", "comfortably", "trek", "humiliation", "rescue", "credited", "dictator", "unusual", "repeatedly", "hervé", "immune", "compromise", "shearer", "trimmed", "alfredo", "legitimate", "lauded", "odd", "stronger", "quirk", "crossbar", "philadelphia", "stressed", "scapegoat", "pop", "exile", "carl", "adamant", "insipid", "twinkle", "contribute", "nicolás", "globalisation", "physio", "presentation", "spying", "counterattack", "electric", "witnessed", "toiling", "lesson", "pierluigi", "collina", "serf", "dominic", "menu", "overdue", "fervour", "lithuania", "barça", "maya", "provides", "conjured", "sheeran", "upward", "disrespect", "clever", "pedri", "reasonably", "raise", "expansion", "scheme", "lane", "coping", "plain", "scale", "laser", "torture", "corridor", "colonial", "equality", "mount", "divine", "tick", "repressive", "unpopularity", "declares", "varying", "relentless", "stretch", "questionable", "fifpro", "heineken", "surprisingly", "moldova", "ring", "sublime", "conflict", "kasper", "tragic", "politically", "rubber", "angst", "isolation", "installed", "prevail", "intensifies", "gifted", "gamble", "indication", "implemented", "permitted", "excitable", "reckoning", "poignant", "hajime", "drowned", "emptied", "golfer", "impending", "luminous", "twilight", "finalised", "congratulation", "carlo", "mum", "realising", "condemn", "eberechi", "defied", "straddle", "shine", "squarely", "involves", "shut", "sufficiently", "relieved", "casual", "esteghlal", "inch", "tended", "clarify", "rejoin", "investigating", "indefinitely", "dignity", "tendon", "isolate", "sporadic", "chat", "endorsed", "prediction", "indulged", "mindset", "rowles", "friendship", "measured", "nov", "louder", "ninian", "whoop", "questioning", "vocal", "bubble", "unopposed", "netflix", "psychologist", "bomb", "youngest", "josh", "cullen", "philippe", "midfielders", "bothered", "fenway", "recreate", "kevan", "upland", "loaded", "peerless", "ieran", "accuracy", "crushing", "bowed", "refers", "legislation", "whip", "proven", "thick", "witch", "claus", "bounced", "turkish", "abandoned", "escalated", "weak", "exposure", "script", "advent", "nearer", "sorry", "dedication", "da", "influenced", "intended", "stewardship", "rope", "goodbye", "jingle", "edward", "copenhagen", "parken", "obsessed", "spfl", "utter", "unfold", "despot", "opinion", "entrance", "brim", "lijnders", "reserve", "driver", "livingston", "hitler", "ootball", "radiotherapy", "daniele", "sister", "circulating", "salvation", "zinchenko", "devotion", "sparking", "palermo", "onana", "brass", "newly", "nayef", "rennes", "speaks", "slow", "businessman", "package", "serb", "seize", "upwards", "division", "controversially", "showdown", "counterterrorism", "protection", "expired", "irony", "vip", "countless", "núñez", "darwin", "tense", "vancouver", "lgbtq", "property", "regulation", "nightlife", "relating", "selecting", "polished", "threatening", "whisper", "dominate", "transporting", "dome", "interviewed", "operate", "partey", "pawn", "permit", "album", "hawkes", "spy", "maintains", "urgent", "romelu", "lukaku", "partridge", "officiated", "contributed", "fibula", "annoyed", "envisaged", "slammed", "reluctant", "mirage", "impressively", "terrible", "f�", "bregas", "analyse", "island", "portion", "klaassen", "insider", "machine", "aziz", "alkmaar", "behich", "ross", "suitcase", "topping", "elneny", "dissent", "sofyan", "tribute", "simplicity", "hijack", "material", "mudryk", "iron", "epcr", "marking", "verge", "ensured", "gate", "lit", "catwalk", "collectively", "imam", "mbemba", "chancel", "violation", "sharp", "admirer", "winding", "floyd", "ruud", "geopolitical", "assisted", "harshly", "sensor", "spun", "aviation", "cosy", "liberalisation", "buddy", "warms", "shrouded", "sadd", "sailing", "accompanied", "overwhelming", "eat", "acceleration", "shout", "canopy", "square", "deaf", "renown", "wonderment", "id", "attempted", "adrift", "imagination", "adviser", "dribble", "schalke", "catalyst", "toured", "unresolved", "rolled", "innovation", "wondering", "mong", "marry", "woke", "recommended", "suited", "crook", "panel", "marred", "rallying", "misplaced", "promoted", "placard", "terminally", "sevilla", "populist", "leeway", "isolating", "brown", "crash", "solidarity", "romantic", "benjamin", "somebody", "inbox", "microphone", "kibue", "affair", "perimeter", "existing", "sake", "drawing", "hop", "affection", "privilege", "originally", "temptation", "expressing", "recognise", "kidnap", "middling", "guidance", "trend", "justin", "remark", "encouraged", "neighbouring", "shilling", "confidential", "parade", "decorated", "volatile", "perfectly", "loser", "reo", "content", "praising", "reigning", "stating", "globally", "sniggering", "skied", "walkout", "milk", "dairy", "staffordshire", "timekeeper", "rousing", "driven", "msc", "galore", "introducing", "district", "reserved", "terrestrial", "leaked", "reyna", "detect", "arose", "raphael", "dealing", "cornerstone", "prompting", "rabiot", "fighting", "tier", "ith", "draconian", "lebanon", "commissioned", "simeone", "starred", "scarcely", "mandarin", "oriental", "uncovered", "ebullient", "charging", "goalpost", "dejan", "grinning", "heavyweight", "crime", "becomes", "inflatable", "freshen", "righteous", "doubted", "illustrate", "dragging", "lends", "touchline", "flutter", "glorious", "humble", "curated", "precocious", "crisp", "christie", "riyadh", "precisely", "topped", "intermittent", "victorious", "rodrigo", "bentancur", "notable", "rejoined", "inquiry", "ruthlessly", "anterior", "withdraw", "spa", "obscene", "hung", "slovenia", "integrity", "establish", "dunmore", "vigilant", "reversed", "trick", "fancied", "object", "bizarrely", "fluffy", "anderlecht", "sportswriter", "ebbed", "bare", "smooth", "shell", "grin", "regarding", "buzzing", "pavard", "scar", "anticipation", "nicely", "watkins", "ollie", "can", "gather", "identifying", "constructed", "spare", "secondary", "frozen", "viral", "killing", "dropping", "panama", "ruptured", "renewed", "massimiliano", "tatchell", "motion", "birdie", "wintry", "welshman", "wieghorst", "assembled", "warhol", "tasked", "duly", "gently", "serbian", "stakeholder", "technician", "easing", "related", "attraction", "intoxicated", "harsh", "kudos", "resulting", "graphic", "clamour", "asterix", "repeated", "valley", "coady", "ire", "jakub", "descent", "pleasantly", "christopher", "suppress", "lounge", "forfeit", "representation", "eca", "serving", "schwarzstein", "realise", "feed", "tumble", "zerbi", "cream", "fraud", "neatly", "appointment", "tucked", "lgbts", "hatelab", "abusive", "awe", "prioritised", "sofa", "mansoura", "deploy", "authoritarian", "teenage", "accredited", "checking", "chesterfield", "romania", "fantasy", "lady", "ian", "vieira", "startling", "desk", "liz", "truss", "hauled", "swooped", "souness", "tennis", "suitable", "tissue", "jump", "quote", "causing", "inflammatory", "iwata", "karaoke", "bounce", "mask", "ticker", "stirring", "capitulation", "mutinous", "takeover", "strobelallee", "stockport", "ahmed", "version", "tycoon", "ludicrous", "eusébio", "jumped", "electronic", "unwell", "blocked", "fluid", "ethnic", "promotional", "tommy", "widdrington", "balkan", "optimistic", "carbon", "matched", "wizardry", "buying", "manages", "chaos", "reflecting", "favourable", "ecstatic", "solution", "ra", "simultaneously", "ushered", "forever", "achieve", "household", "mahabad", "insightful", "daring", "richest", "relevant", "election", "pinged", "gym", "azzurri", "clinching", "insisting", "axel", "scout", "embrace", "trinidad", "mccall", "suspicious", "negotiate", "check", "issued", "aching", "rishi", "sunak", "diagnosis", "serie", "salem", "albert", "index", "yell", "patrick", "boisterous", "mathias", "summoned", "orgy", "thrust", "bt", "bp", "eldest", "landlord", "diogo", "quinn", "sceptical", "equity", "journalism", "eager", "hm", "lap", "gif", "medina", "manned", "phenomenon", "steady", "chatted", "bouncing", "flooding", "lavish", "reveal", "snuff", "thrashing", "decorum", "unleashed", "creature", "sponsored", "charity", "waistcoat", "maintaining", "terror", "lenient", "dedicated", "almighty", "pumping", "underground", "casino", "plausible", "roughly", "downpour", "stray", "unaware", "poster", "kelly", "acrobatic", "replacing", "upcoming", "trossard", "chisora", "parc", "hollow", "farewell", "forgotten", "hacking", "guimarães", "addressed", "perception", "coincidence", "broadly", "float", "relocated", "transformative", "parallel", "clinch", "theory", "automatic", "muani", "randal", "kolo", "kimpembe", "presnel", "bourdeaux", "laporta", "noppert", "andries", "remembers", "jest", "deployed", "limitation", "lautaro", "joseph", "giovanni", "indelible", "connected", "ethic", "enlisted", "ill", "provincial", "posed", "baking", "ownership", "hummel", "toned", "asa", "coleen", "cheryl", "boateng", "jackie", "monaco", "assault", "kyprios", "coltrane", "quit", "exasperating", "sexuality", "slide", "francesco", "totalling", "export", "matthijs", "typhoon", "belong", "mighty", "dayot", "wojciech", "encapsulates", "weather", "protesting", "continuing", "radiation", "sanctuary", "contractor", "starter", "potato", "andrea", "mahsa", "arcadia", "being", "abbey", "greenyards", "promising", "craig", "cracking", "pressed", "memo", "na", "disgraced", "despair", "anna", "wintour", "vogue", "employee", "pleasantry", "videographer", "orking", "notably", "goalkeeping", "pritchard", "elderly", "riverside", "medicine", "nazi", "slim", "fatboy", "legend", "transatlantic", "vein", "breathes", "custom", "einstein", "un", "jair", "optimism", "keegan", "selling", "drove", "commission", "narrator", "engaging", "resource", "forcing", "chest", "ella", "alan", "leo", "promoting", "donald", "principal", "pouring", "congregation", "mourn", "infection", "confirmation", "danilo", "grace", "maitlis", "goodall", "sopel", "pinnock", "norgaard", "obstacle", "overpriced", "wright", "saatchi", "garb", "imitation", "submitted", "scrambled", "fighter", "metalist", "sixty", "frustrated", "riyal", "tory", "inner", "underneath", "economic", "posh", "apologised", "fulfil", "aleksander", "influencer", "carter", "basically", "seventeen", "blizzard", "jinping", "spirit", "kobayashi", "yuki", "researcher", "article", "enner", "accident", "proposing", "wondered", "raphinha", "fascinating", "regardless", "enhanced", "demanding", "afc", "tenth", "grandfather", "virtue", "google", "supplied", "ridicule", "edited", "todd", "boehly", "soundbites", "investor", "boufal", "bernardo", "enabled", "donelan", "writing", "penchant", "timo", "evolved", "leap", "salary", "joking", "breached", "monster", "nusret", "fitting", "revelation", "rogue", "dog", "arguing", "eligible", "padel", "oversee", "rushed", "soviet", "pleasing", "surplus", "lbc", "voting", "targeting", "stripped", "byron", "ineligible", "famously", "lioness", "shi", "scarlet", "junior", "purely", "roman", "society", "whenever", "pursuing", "entrepreneur", "rome", "ack", "exploited", "laura", "rimet", "suspected", "user", "abandoning", "ederson", "fife", "kincardine", "handcuffed", "vehicle", "grim", "jason", "ewen", "snatch", "peeling", "yamashita", "kathryn", "salima", "nesbitt", "yoshimi", "karen", "neuza", "staring", "preferred", "profit", "luiz", "funding", "structure", "pronounced", "macintyre", "fitzpatrick", "felled", "swarm", "venture", "grabbing", "minority", "condemning", "crcc", "xinjiang", "tip", "chic", "msheireb", "convicted", "prolonged", "dutchman", "apology", "keating", "ronan", "drifting", "procurement", "fenwick", "lord", "dos", "persecution", "essex", "code", "ocean", "llywelyn", "nearby", "jockey", "foundation", "bedroom", "lesbian", "valera", "ideal", "mukansanga", "station", "perk", "anouar", "aligned", "manufacturer", "tello", "railway", "ell", "genuinely", "pushing", "enthusiastic", "immigration", "attends", "adult", "tartt", "kissing", "comprehensive", "bemoaning", "rely", "positivity", "indulge", "glad", "context", "marble", "maelstrom", "plunged", "hack", "exotic", "traveller", "underestimating", "likewise", "unity", "disrespectful", "emphasising", "credibility", "cascarino", "bang", "apparently", "shutdown", "pittance", "closure", "regulator", "unavoidable", "culminating", "roaring", "stern", "underway", "sabdi", "persian", "sunset", "unifier", "characteristic", "exuberance", "mistreatment", "scrutinised", "animosity", "pitched", "incomprehensible", "tackle", "shiny", "lining", "surviving", "trademark", "reply", "unbelievable", "familiarity", "blank", "impotent", "pryce", "collated", "competed", "bullish", "telephone", "entrust", "loudest", "blower", "overjoyed", "stared", "consult", "populated", "shutting", "expands", "sentiment", "collusion", "infamously", "wrestling", "nepal", "inferior", "skilful", "exhilarating", "impatient", "sensible", "grabbed", "nly", "probable", "winless", "indecisive", "residence", "triangulate", "onlys", "lapse", "sadness", "respectfully", "diminish", "tented", "dutiful", "bst", "technically", "breeze", "extortionate", "shifted", "provision", "confirming", "shift", "ashley", "ward", "dented", "danny", "hence", "reminds", "birdhouse", "fiendish", "connoisseur", "musician", "enthusiast", "bullshit", "objecting", "contain", "problematic", "lighter", "rihla", "absent", "moldovans", "overreacting", "whipping", "thumping", "drift", "surreal", "onslaught", "deluge", "infuriate", "reviewing", "engulfed", "embroiled", "indicted", "traveling", "mounted", "hendo", "facetime", "lallana", "rang", "complaining", "listened", "longingly", "trim", "impressing", "unfazed", "competitor", "benign", "bureaucracy", "rematch", "locally", "thanking", "extensive", "caicedo", "moisés", "teary", "irks", "zlato", "morelos", "barren", "poking", "banter", "nicholson", "qualifies", "dictatorship", "necessitated", "citizenship", "monarchical", "disrupting", "client", "consoled", "haunted", "sarri", "maurizio", "ruben", "indispensability", "centrepiece", "inadvertently", "mariner", "overrun", "enjoyable", "beatable", "eing", "benefited", "stunned", "appreciative", "haunch", "bearable", "useful", "openly", "budget", "tougher", "annoy", "enables", "rash", "pit", "superpower", "whetted", "lingering", "hide", "bitterly", "reckless", "ramin", "rezaeian", "prix", "grand", "polite", "daunted", "transmitted", "composed", "plotlines", "licensed", "collided", "trooped", "disconsolately", "relayed", "abusing", "understandable", "sticking", "glee", "misfortune", "ook", "nemesis", "pilgrimage", "spirited", "magician", "reviewed", "retain", "servicing", "begging", "doorstep", "demonstrate", "hristian", "rethink", "mourning", "decriminalisation", "nod", "married", "wll", "jarred", "brunch", "raging", "frontier", "stubborn", "agrees", "composite", "elevation", "flowing", "tasty", "knife", "unbeatable", "inserted", "alter", "presently", "truman", "inconsistency", "punished", "powered", "naive", "rallied", "partisan", "alphabetically", "creator", "categorically", "ince", "paraguay", "kuala", "lumpur", "renard", "afield", "shehri", "persuade", "overcoming", "flex", "aftermath", "reckon", "spectacularly", "solidly", "pleasant", "buck", "jazeera", "informing", "muscular", "acknowledging", "arrangement", "aggrieved", "unhappiness", "expand", "routinely", "generated", "declaring", "frenzy", "smattering", "tyrick", "undefeated", "tight", "kabak", "settle", "ozan", "syrian", "temperament", "glancing", "scouring", "absentee", "soho", "outdoor", "co", "boiling", "glamour", "twist", "prognosis", "sparkling", "immigrant", "dein", "pressing", "contributing", "twinge", "glum", "faint", "clawed", "inevitable", "walkover", "satisfying", "unpredictable", "resound", "collection", "snazzy", "sticker", "manifest", "bling", "hideousness", "repurposing", "sweepstake", "panini", "futuristic", "grow", "dissect", "herzegovina", "bosnia", "pin", "leaderboard", "minded", "tiraspol", "dispelling", "forensic", "guessing", "devised", "erred", "clint", "dempsey", "sample", "employ", "trotted", "mirror", "elected", "packing", "frightened", "gauge", "coventry", "depending", "lurch", "caravan", "deepened", "tournamnet", "contending", "brighter", "symptom", "norm", "vouch", "extrapolate", "orderly", "concluding", "trumpeted", "honing", "prosecute", "discriminative", "timeline", "floodlit", "revisited", "putt", "bloodless", "tantalising", "undulating", "ascendant", "justified", "guttural", "echoed", "soler", "fez", "matured", "circus", "draped", "sparing", "doubtless", "consolation", "salisbury", "frost", "instantly", "terracing", "recruited", "ray", "grunt", "lull", "cheerily", "moaning", "unanimous", "disapproval", "boasted", "presto", "demolition", "brightest", "wingback", "marquee", "borrowed", "cramp", "ovation", "disrupted", "appalled", "arrange", "milkshake", "objective", "registering", "solanke", "supremacy", "distinct", "slog", "mingling", "airbase", "finalissima", "royalty", "grandchild", "prestwick", "elvis", "brush", "presley", "checked", "allchurch", "emergence", "rewrite", "ivor", "kelsey", "folklore", "dissuaded", "protested", "closeted", "misunderstood", "prodigy", "terrorising", "abundance", "cooled", "proximity", "cape", "safely", "stupidity", "duck", "totally", "tepid", "napoli", "paco", "genk", "pacey", "gomez", "kouli", "contemplated", "punishes", "density", "unrelenting", "catalan", "distribute", "reprised", "proposition", "clearer", "intensity", "degree", "lacking", "lazy", "lesser", "muddled", "painted", "fulfilment", "heel", "conquer", "attire", "everest", "towered", "revived", "capture", "bewildered", "statue", "lying", "thumb", "shoe", "decimated", "deafening", "bellowed", "brotherhood", "joyride", "garfield", "exist", "blush", "decisiveness", "rtl", "bout", "jonathan", "barnett", "ukranian", "postpone", "contesting", "tribune", "photographer", "eluded", "cite", "constable", "emerging", "cheshire", "edged", "advises", "asylum", "incendiary", "outlawing", "racked", "disciplined", "austrian", "cleared", "colourful", "glaring", "handing", "unstoppable", "deflating", "fright", "bewilderment", "hurtled", "acquire", "gobsmackingly", "aback", "mechanism", "goede", "hockey", "essay", "intricately", "layoff", "anxiously", "crutch", "whichever", "distraction", "momentary", "howl", "dire", "reprisal", "disloyalty", "brilliance", "aptly", "bravery", "merciless", "rampaging", "unthinkable", "sternest", "sizzling", "montage", "predominantly", "boasting", "printed", "expat", "chorus", "handful", "mooy", "onerous", "scaled", "overloaded", "gobbling", "suffocating", "withdrawing", "announces", "harangued", "cardigan", "decency", "continental", "slated", "conversely", "pale", "vicarious", "comparison", "fare", "fluctuation", "pratfall", "vagary", "haphazardly", "bucking", "bronco", "lasted", "input", "peaked", "streak", "command", "outset", "freshly", "greenery", "recurrence", "pulse", "alley", "unwavering", "slew", "rance", "christophe", "flop", "unconvincing", "sole", "ignominiously", "rife", "jallet", "colonialism", "remote", "slept", "crowded", "scaffolder", "chandra", "rup", "rumba", "shade", "shelter", "realistically", "exploiting", "grudge", "sterile", "prolong", "downplayed", "cobham", "easyjet", "moonscape", "forth", "baffling", "scree", "studded", "superfluous", "disneyland", "shuttling", "laughed", "merthyr", "damp", "tweaking", "rested", "overtake", "discomfort", "output", "extraordinarily", "provoked", "chicanery", "arcane", "awful", "pernickety", "scrape", "eyebrow", "disallowed", "knowingly", "hoisting", "norton", "presenting", "rerun", "eurovision", "disturbing", "undercover", "creepy", "zip", "youthful", "rounded", "mouse", "petulant", "treasure", "cynic", "beck", "arc", "goldenballs", "foray", "owing", "uplifting", "halo", "scare", "unfancied", "opportunistic", "lacklustre", "vindicated", "quickfire", "icing", "juncture", "wounded", "preliminary", "maiden", "twittersphere", "pantomime", "bronze", "facilitate", "benton", "petrakov", "lviv", "malta", "averse", "outweighed", "understatement", "paired", "implied", "burgled", "sympathetic", "complimentary", "indebted", "barnstorming", "wahhabi", "islam", "warn", "observed", "smallest", "sunni", "thermometer", "fallout", "sailed", "freezing", "fatigue", "taging", "overcomes", "fold", "heady", "invigorated", "adrenaline", "roca", "jesse", "marsch", "stateside", "virtually", "hereford", "revelling", "worldlier", "cocoon", "hree", "lippi", "marcello", "constantine", "intrepid", "englishman", "convivial", "suzhou", "argued", "dying", "poorly", "hoffenheim", "byline", "raum", "transitional", "outnumbered", "interminable", "welve", "glamorous", "mercifully", "sheikhdom", "committing", "tentatively", "tempo", "horse", "characteristically", "bond", "ick", "tinkering", "counting", "tock", "turbulent", "curiously", "detached", "heartbeat", "omorrow", "premature", "staggering", "blaming", "intelligent", "upbeat", "turin", "oversaw", "frenchwoman", "constitution", "mina", "concourse", "skirted", "patriotism", "glenn", "hoddle", "bladder", "yerry", "scope", "broaden", "mercy", "runway", "stalk", "respite", "angling", "hammered", "banking", "eloquent", "covertly", "wilton", "tiresome", "affinity", "dilute", "hindrance", "terrace", "eter", "ginger", "crouch", "beanpole", "stall", "casablanca", "swamped", "chairwoman", "debbie", "soundtrack", "sinatra", "carrot", "balogun", "dangled", "flo", "carriageway", "mishap", "gambled", "shy", "underwhelming", "external", "scoffed", "strives", "inflicted", "draping", "exchanged", "timed", "candid", "signage", "volunteer", "pendant", "backstreet", "advise", "sweating", "jar", "siro", "threatens", "dialogue", "advocate", "lotte", "lobbying", "frontrunners", "efficiency", "mess", "compass", "admiration", "natured", "detritus", "bearing", "voicing", "wrench", "quantify", "citing", "shaken", "capitulated", "rooted", "berated", "embarrassed", "jahanbakhsh", "alireza", "unsettle", "motivated", "veiled", "essence", "barnet", "xenophobic", "schadenfreude", "exaggerated", "auld", "siege", "recognises", "masia", "boudiaf", "afif", "khoukhi", "hatem", "félix", "kish", "waiving", "engrave", "vault", "peer", "inscription", "classy", "nurtured", "meticulously", "rio", "surrendering", "bragging", "licking", "prestige", "sleeping", "groove", "unequivocally", "ferociously", "podium", "petrol", "shrewd", "superstitious", "connection", "trusting", "wavelength", "scrapped", "justification", "despotic", "pretending", "gesturing", "principled", "indefensible", "prized", "prescient", "atlas", "combativity", "slight", "thirty", "courtois", "constitute", "ingenuity", "involve", "adventurous", "grind", "nostalgic", "poorer", "aesthetically", "teammate", "resolved", "define", "cement", "drafted", "ifa", "forefront", "hip", "denies", "termination", "pro", "viable", "germnay", "replicate", "revitalising", "stale", "scotsman", "ragged", "jarring", "embossed", "pencil", "cliff", "memento", "skyscraper", "occupational", "illustrated", "readiness", "assemble", "entertainer", "crucially", "exemption", "nglish", "tabloid", "disgraceful", "charlatan", "gaiety", "tacit", "advising", "arrives", "ralph", "aggravated", "procedure", "gerardo", "raúl", "skip", "rehab", "martino", "lobby", "respective", "resumption", "reuniting", "shaping", "tomiyasu", "takehiro", "fickle", "bruges", "hostage", "feted", "nominee", "evaluate", "hastily", "petersburg", "slows", "reneged", "skin", "sually", "pressured", "taunt", "lottery", "snarl", "otamendi", "discard", "unheralded", "gonzález", "lópez", "katie", "topless", "sweetheart", "minimise", "goodland", "overtaking", "televised", "steer", "danced", "stylistic", "flintstone", "costume", "flashback", "assistance", "burglary", "unseen", "memorabilia", "childish", "rambouillet", "clairefontaine", "athletically", "unearthing", "succeeded", "strangest", "gorge", "cavalcade", "sumptuous", "buffet", "metropolis", "ferried", "rolling", "smallish", "undeniably", "cauldron", "poxy", "heartfelt", "nonsense", "admire", "disparaged", "utterance", "expansive", "fishing", "merry", "prodigious", "giuseppe", "hooked", "majesty", "dismay", "despairing", "macleod", "acclimatisation", "researched", "stifling", "caroline", "carthage", "dane", "bracing", "multitude", "seal", "eagle", "kinship", "servitude", "kindled", "thames", "encompassed", "darkness", "adversity", "benin", "ouaddou", "abdeslam", "advocacy", "collapse", "grumpy", "totalled", "masterclasses", "siesta", "diambars", "tiredness", "pour", "brit", "imbledon", "blameless", "rage", "minuscule", "blithely", "craven", "sail", "charming", "muted", "burger", "mixture", "twang", "oxygen", "moon", "cow", "exiting", "suspense", "unison", "unite", "ngo", "poorest", "fairsquare", "pearl", "concludes", "anticipates", "drained", "prioritising", "arch", "sorted", "substandard", "scuppered", "littered", "lend", "ascendancy", "murder", "strap", "enfield", "sprawling", "huddling", "notebook", "southern", "easygoing", "fanatic", "yowl", "lashing", "hurrying", "glove", "dislocated", "chamber", "pasok", "soldier", "sharq", "khomeini", "rotating", "uphill", "fresher", "connect", "bisexual", "harassment", "transvestism", "therapy", "multipolar", "demographic", "plant", "prosper", "polycentric", "selective", "economics", "attractive", "unfettered", "saliba", "rusty", "baggage", "cited", "anticlimactic", "indiscipline", "ignominy", "weighed", "iceland", "dissimilar", "spied", "intertwined", "burdened", "scarred", "poison", "disturbed", "punditry", "troubling", "io", "spilling", "poise", "helm", "madder", "goosebump", "pump", "santa", "stumped", "fairy", "emmerdale", "express", "plug", "miffed", "chorzow", "jaunt", "ibrahimovic", "zlatan", "distracts", "basic", "brotherly", "fatigued", "timetable", "privately", "lest", "elegant", "lilian", "mönchengladbach", "tuta", "djed", "workhorse", "numerical", "clavijo", "tallest", "ayne", "outfielder", "sofia", "endless", "hardcore", "wrap", "shone", "rebound", "retaken", "tipped", "montpellier", "lecomte", "tenerife", "briefing", "impeccable", "fondest", "faded", "terrified", "ric", "cruellest", "envied", "creativity", "shareholding", "revive", "intrigue", "compile", "array", "columnist", "simpler", "furlong", "pistoleiro", "dink", "blast", "occurrence", "gonçalo", "mayhem", "sacred", "ripped", "inhibited", "formation", "undoubted", "restricting", "gunghos", "reprehensible", "juggle", "yelled", "tribulation", "hatful", "selfish", "pqq", "sitter", "signify", "totality", "admission", "fortitude", "actively", "practise", "arises", "bribing", "assures", "insignificant", "orchestrated", "heist", "cycle", "kigali", "statute", "clattenburg", "stephanie", "insufficient", "cracker", "smiled", "amusingly", "memorably", "guaranteeing", "lecture", "condemnation", "jumper", "smartphones", "dreamland", "backpass", "disintegrate", "disorientated", "silence", "starring", "inexcusable", "dignitary", "pedigree", "casket", "appallingly", "crassness", "disabled", "gaffe", "vanity", "unanimously", "concacaf", "pall", "conflicting", "ingredient", "jock", "stein", "expanse", "apace", "continual", "gleaming", "rejuvenated", "wipe", "forrest", "extinct", "turnbull", "greg", "deletion", "trivial", "constraining", "trifling", "pressuring", "unfolding", "sochi", "gaze", "pang", "intuition", "envy", "matuidi", "prompt", "composition", "jealousy", "blaise", "tongue", "drank", "whimper", "dreamt", "idrissa", "fratton", "gueye", "steel", "cisse", "fortunately", "bleat", "inherit", "namely", "nuance", "constructive", "prejudice", "veered", "dismiss", "regrettable", "trope", "misinformation", "commented", "arduous", "wsl", "barclays", "rebranded", "defy", "dig", "epicurean", "influencers", "kendall", "payne", "jenner", "describing", "bisht", "inconsolable", "eighteen", "stored", "heal", "vile", "globe", "fascinatingly", "pepper", "stipulate", "goodison", "acrimonious", "unveiling", "dug", "cessation", "onset", "stoical", "crackdown", "ramezanpoor", "internet", "aliasghar", "exertion", "greave", "moura", "gil", "bryan", "equalling", "indian", "amiably", "encountered", "ate", "warmed", "confronting", "rude", "flaky", "hungry", "bruised", "ot", "confiscating", "disparity", "extending", "distinguishable", "noon", "cobweb", "expunging", "recharge", "blew", "stricken", "retook", "die", "bend", "alchemy", "uncontainable", "finely", "tandem", "volleying", "darting", "more", "violating", "underscored", "fend", "zdf", "khalid", "fringe", "weakest", "updating", "fairly", "scorpion", "scouting", "blackpool", "accepting", "fashanu", "deflected", "captaining", "fearing", "autograph", "almeria", "gerard", "croydon", "thunderclap", "heartwarming", "celand", "icelandic", "revolve", "abiding", "illuminating", "atmospheric", "muezzin", "underestimate", "watcher", "febrile", "atop", "gridlock", "veer", "ramp", "sealine", "forklift", "filipino", "retained", "protracted", "dip", "reception", "ogbene", "michy", "rotherham", "chiedozie", "handball", "asset", "concentrating", "spared", "playoff", "essential", "brew", "packet", "sourced", "chocolate", "twelve", "justify", "beaming", "ron", "superintendent", "stalled", "elevated", "mischief", "miniature", "thwart", "comedown", "masterpiece", "imperious", "flaw", "foible", "nostalgia", "ironed", "fakery", "moonlight", "mountainous", "bless", "othing", "bathed", "rubble", "sipping", "answering", "flooded", "calmly", "espresso", "racially", "creation", "tall", "abused", "occupant", "stole", "oxshott", "jewellery", "gated", "burglar", "alerted", "fierce", "partnership", "harness", "volodymyr", "assurance", "wealthy", "investigative", "reveals", "database", "aqarone", "mushref", "estate", "lent", "chose", "impossibly", "hich", "cowed", "lameness", "toppled", "brandishing", "hoisted", "performs", "fantastically", "riptide", "statistical", "fulsome", "empirical", "erhaps", "purred", "staunch", "recede", "troubled", "prohibit", "morally", "preferable", "shocking", "dribbling", "teased", "popularity", "fulfilled", "seductive", "undiminished", "resonance", "exceed", "bolton", "clocked", "underwhelmed", "ups", "down", "backlash", "amazon", "noting", "pontificating", "worcester", "realpolitik", "superhuman", "countryman", "boyo", "happiest", "universe", "greeting", "gunter", "karmic", "snatching", "jaw", "iplayer", "woe", "biography", "writes", "ted", "honourably", "fails", "variable", "fooled", "reassurance", "damaged", "nisbet", "maloney", "preclude", "cv", "framed", "coolness", "composure", "yassine", "underscore", "recreating", "sharply", "rhythm", "plodding", "outfield", "exuberant", "incessant", "justly", "bagged", "wrestled", "clarity", "morteza", "penalised", "pouraliganji", "grappling", "waved", "ruggeri", "plea", "oscar", "slater", "monchengladbach", "maxi", "resigned", "fanfare", "enthralling", "et", "insouciance", "gallic", "gentleman", "ing", "merchant", "breakdown", "onny", "berkshire", "wilkinson", "companion", "backside", "ticking", "ere", "gasp", "generally", "accomplishment", "orwell", "clichéd", "rehash", "novichok", "dosh", "morality", "plc", "bunged", "aeroflot", "sightseeing", "rearrangement", "ashore", "moored", "renting", "harbour", "revolution", "anas", "zaroury", "demonstrating", "benson", "replay", "embarrass", "presided", "stumbled", "unreasonable", "unrecognisable", "wholly", "stewing", "reckons", "melbourne", "clarifying", "horrifying", "ordeal", "invader", "robbery", "memorandum", "outlining", "glance", "assorted", "reflective", "youssoufa", "tracked", "underline", "submit", "fading", "doggedly", "majestic", "mustered", "sank", "strasbourg", "sufficient", "unnamed", "befalling", "decrying", "venality", "romesh", "welling", "ranganathan", "talbot", "serenading", "jubilant", "seasonal", "segued", "trigonometry", "discourse", "seeded", "unlucky", "vllaznia", "eventful", "tumultuous", "bakhmut", "professionally", "photographed", "paramount", "proof", "marathon", "contained", "concept", "puskas", "detour", "budapest", "afterthought", "contend", "gruelling", "pointless", "ceo", "swollen", "nervously", "eh", "teetering", "bruce", "fat", "boss", "liner", "espanyol", "plough", "donated", "unicef", "bye", "towering", "seaman", "immortal", "perish", "condensed", "dizzying", "unparalleled", "runner", "contestant", "notch", "unpalatable", "parachute", "ramped", "replete", "survival", "rider", "rupture", "accompanies", "bernabéu", "foresaw", "xavi", "signalling", "peered", "arcelona", "angle", "sprinter", "clocking", "oregon", "hq", "nethaneel", "tilted", "admired", "bonucci", "spinazzola", "resemble", "sicily", "mancini", "flush", "nahuel", "arced", "molina", "reverse", "shrunk", "gabon", "irregularity", "rightful", "sonda", "supermarket", "delcir", "dis", "internationale", "keener", "wealthier", "healthier", "profitable", "stalemate", "begovic", "suffers", "merseyside", "asmir", "deputise", "damaging", "denying", "mcpake", "improbability", "duke", "janoub", "mere", "nordic", "dissatisfaction", "follows", "squeezing", "conjecture", "concertina", "nodded", "zimmerman", "fiercely", "clumsily", "equalise", "dispatched", "savvy", "scott", "guarding", "enforcing", "upsetting", "dictate", "formidable", "sofri", "sagna", "teranga", "distinctive", "sparkle", "curve", "frenkie", "pacy", "jong", "lay", "stoke", "luton", "fuse", "delivers", "scripted", "sealing", "caledonian", "supposedly", "hallelujah", "thistle", "mug", "millwall", "encapsulating", "fanatical", "thumped", "inverness", "santiago", "alcalá", "paratrooper", "henares", "cogedor", "communicated", "celia", "trekker", "undertone", "embedded", "belting", "dimension", "overtone", "hate", "innocent", "fracking", "dear", "slashing", "funded", "whitehall", "movement", "melli", "remake", "scotty", "massage", "comedy", "screwball", "blockbuster", "stressful", "compston", "parlour", "hilarious", "jetlagged", "kickabouts", "soared", "furnishing", "reintroduce", "discouraged", "suisse", "gasfield", "ridiculously", "texan", "madelaine", "unkets", "cleveland", "taskforce", "instance", "riddled", "anxiety", "culminated", "chanting", "jonny", "batch", "faulty", "reparation", "mercurial", "forthright", "sever", "conventional", "brennan", "helsinki", "eintracht", "flanked", "hi", "workload", "indoors", "xherdan", "overhead", "basel", "oppressive", "reacted", "aggressively", "moralise", "fatma", "pledging", "releasing", "pleading", "sensitive", "suspending", "tackling", "worthy", "woolly", "succumbed", "maturing", "buggy", "unlock", "backroom", "forensically", "stabilise", "limping", "determine", "joie", "saturnine", "vivre", "flak", "héloïse", "christine", "redcar", "guise", "trans", "acknowledgment", "unwillingness", "smart", "lynam", "survey", "mawson", "consultancy", "boom", "bangladesh", "guardian", "equivalence", "valid", "barnes", "moreover", "severity", "weah", "emphatic", "salvaged", "timothy", "rued", "crush", "dispatch", "atomic", "overshadow", "tactically", "kitten", "outwit", "madly", "hindsight", "preceded", "nadir", "debutant", "ricardo", "tobago", "grange", "russell", "execute", "pippa", "clashed", "goading", "eventual", "bodychecking", "flourish", "paredes", "pattern", "repression", "censorship", "hereditary", "junta", "urban", "dukhan", "mmc", "located", "chugged", "workman", "dhow", "emblem", "adjustment", "gharafat", "etat", "sûreté", "spook", "interference", "stash", "blackburn", "paperwork", "andré", "brien", "cooper", "calligraphy", "disputed", "sanity", "insanity", "faculty", "psychology", "regained", "anointed", "throne", "formally", "trio", "heir", "persuasive", "bemusing", "jingoism", "aloft", "bangladeshi", "actual", "coursing", "dhaka", "chittagong", "gonfalon", "promenade", "holy", "antisemitism", "throttle", "outcast", "swanky", "bypass", "stamp", "loughborough", "orr", "madeleine", "ecology", "chilled", "sprayed", "reshaped", "inflict", "clinical", "gleamed", "inconvenience", "shivering", "toasted", "ringside", "practical", "resembled", "unexpectedly", "unwitting", "tested", "beadle", "shanghai", "instantaneous", "furthermore", "tracking", "manually", "montreal", "embarks", "croat", "understandably", "contemplate", "precautionary", "chase", "fleeting", "predator", "graced", "choreographing", "seesaw", "momentum", "authenticity", "fabulous", "harmonious", "occupy", "uniquely", "tremendous", "maintain", "freebie", "strangely", "varied", "harmony", "antidote", "sprinted", "comforting", "ude", "haunt", "regroup", "nagging", "teddy", "ayman", "handcuff", "cocaine", "meow", "smashed", "tamed", "carnage", "lorenzo", "sculptor", "forming", "marine", "bidda", "flamboyant", "guiding", "reviving", "grammy", "witnessing", "obsequiousness", "haul", "subjective", "kiwi", "envious", "steam", "treasured", "ascended", "boyhood", "translates", "departing", "afar", "donating", "withheld", "santosh", "unpaid", "fore", "repaid", "terminated", "influx", "paltry", "incurred", "downloaded", "battery", "hayya", "afonso", "mri", "stripe", "backfiring", "inability", "loose", "blood", "consternation", "jamón", "expedition", "scaffolding", "pork", "importation", "tonne", "el", "país", "prohibited", "pitchside", "bemoaned", "endure", "imported", "reservation", "repair", "édouard", "historical", "steal", "spill", "oleg", "breakout", "dubious", "unofficial", "toto", "schillaci", "elevate", "pedestal", "wielded", "salenko", "ah", "unstable", "heartening", "mcneil", "dwight", "aribo", "armstrong", "napa", "linekers", "humour", "dealt", "muddy", "ayia", "impeccably", "jungle", "rebuilt", "eerily", "dyslexia", "kelle", "dyslexic", "articulate", "straightaway", "roos", "gascoigne", "penultimate", "clamouring", "copied", "proverbial", "handbrake", "yielding", "butt", "printing", "punchline", "jibe", "cuba", "mockery", "gleefully", "legislative", "beset", "legalise", "acknowledge", "exploitation", "motivating", "payroll", "crave", "current", "valiant", "streetwise", "artful", "dishonourables", "wanda", "intimidation", "sly", "metropolitano", "flinch", "togetherness", "bore", "obscure", "petty", "fume", "pining", "bloke", "heathrow", "invented", "clipboard", "chill", "bureaucrat", "nowadays", "experiment", "lawmaking", "relay", "organising", "bowler", "subsidised", "telly", "attach", "queuing", "preferably", "vest", "wimbledon", "melodrama", "breathing", "peach", "belarus", "diverting", "garibaldi", "bellini", "starved", "wagdom", "nyone", "wagatha", "danyel", "reiche", "analysing", "hesitating", "lecturer", "slush", "brannagan", "georgetown", "slowness", "assessment", "embracing", "ue", "diversity", "mihir", "dived", "darted", "exeter", "misfiring", "ark", "listening", "exempt", "considerable", "obscenely", "ambassadorial", "giddy", "loudly", "armando", "kempes", "mario", "valdano", "viva", "conmebol", "strained", "pierre", "sorbonne", "healed", "exacerbate", "braced", "vermeren", "protectorate", "framework", "denouncing", "revenge", "meniscus", "sobol", "allowance", "eduard", "alves", "spearhead", "dani", "unam", "mentioning", "dim", "livermore", "jermain", "bertrand", "jake", "dembélé", "precise", "ousmane", "defoe", "cahill", "skirmish", "bastard", "sleet", "stormy", "steeped", "milburn", "fought", "oval", "print", "uncompromising", "hughes", "sepia", "leighton", "taught", "handling", "laffan", "tornado", "hailstone", "gavi", "lebanese", "horrified", "deceptive", "conductor", "heath", "haywards", "stealing", "barbaric", "prestigious", "modernising", "puffed", "strutted", "bacca", "levelled", "chequered", "snub", "mainichi", "shimbun", "torrent", "clown", "unloaded", "cracked", "warmth", "goon", "luzhniki", "smirked", "continuity", "unspecified", "respected", "imminently", "bolstered", "frightening", "whitewashed", "pave", "disrupt", "riot", "shrine", "desperation", "shiraz", "gunman", "gnaws", "chalked", "swift", "combination", "blossoming", "messier", "necessarily", "grievance", "outbreak", "appalling", "wheeled", "aphorism", "recycled", "archive", "capello", "dried", "arid", "fabio", "notion", "reattributed", "assumption", "trophyless", "warmly", "immersion", "tangerine", "autocracy", "wheeze", "preventing", "laundering", "playbook", "scrubbed", "freely", "neuter", "banning", "tellingly", "convoy", "vvips", "gridlocked", "jam", "escorting", "pepsi", "artfully", "samsung", "dwarfed", "structured", "myriad", "haig", "supercharge", "earns", "controlling", "authentic", "beckhams", "banked", "biennial", "nominate", "dfb", "criterion", "disruption", "conditional", "mertens", "witzel", "foisted", "vertonghen", "thinker", "tire", "relentlessly", "alderweireld", "dread", "toby", "jan", "rung", "opta", "sec", "vacancy", "engrossed", "campbell", "sol", "lma", "ahora", "soloist", "falklands", "choir", "no", "develops", "coined", "volvimos", "roxburgh", "muchachos", "ilusionar", "impediment", "socket", "bystander", "fractured", "displaced", "salivate", "protecting", "durham", "protector", "inscribed", "est", "timendum", "humid", "nihil", "latin", "charismatic", "peacekeeper", "blade", "apologetic", "pickle", "tricky", "entitled", "coincide", "notoriously", "fever", "knottiest", "naming", "fff", "allayed", "lightly", "immediatly", "hobbled", "tropic", "prickly", "exclusive", "ousted", "skating", "jot", "truus", "shorthand", "rein", "algarve", "outskirt", "ravel", "morrison", "hive", "powerhouse", "statistically", "accurately", "measuring", "thirdly", "timekeeping", "firstly", "calculation", "secondly", "inappropriate", "adulterer", "prosecution", "attendee", "unreliable", "recession", "forgiven", "statistic", "intending", "stalling", "boulaye", "dia", "equalised", "egyptian", "engulfing", "propping", "sickening", "voted", "fomo", "preening", "grift", "carping", "trousering", "grubby", "objection", "principle", "calculating", "cynicism", "balderdash", "etc", "outdone", "bluster", "bluff", "retail", "terribly", "cloying", "scepticism", "trusty", "investigate", "culprit", "arly", "thunderous", "chased", "cartoon", "supercar", "stocky", "chaser", "revved", "nightclub", "purr", "stretched", "parasite", "banc", "centred", "homecoming", "holler", "unveiled", "complemented", "permutation", "choosing", "outlined", "overseeing", "implement", "communication", "contentious", "refereed", "transparency", "mukasanga", "midwife", "insistence", "instructor", "pioneer", "grander", "televises", "spotlighted", "espite", "silent", "lbgtq", "defiance", "captured", "bravado", "gig", "assiduously", "jazz", "privacy", "rendition", "cabaret", "stylised", "consists", "erupted", "breaching", "allah", "horizontal", "offending", "disgrace", "gamesmanship", "resign", "abdullah", "uproar", "ansari", "bury", "harm", "curtis", "assisting", "bull", "fes", "talismanic", "kirby", "alessia", "pondered", "comprehensively", "outplayed", "ellen", "mead", "beth", "fran", "russo", "chloe", "bouquet", "barbed", "rey", "del", "wire", "subscribe", "click", "pinch", "weekday", "gheerbrant", "tickbox", "updated", "concise", "pillar", "effortlessly", "wonderfully", "criticise", "loath", "ürgen", "sympathises", "doubting", "juggled", "fk", "discarded", "viking", "bumpy", "unconventional", "eclipsed", "yeovil", "ieffer", "lifeguard", "perseverance", "convert", "macclesfield", "panic", "grandest", "salubrious", "wavertree", "cousin", "shocked", "grandparent", "abdoulaye", "trepidation", "scarce", "lumberjack", "unconditional", "extension", "horn", "beckoned", "motahhari", "witness", "clamped", "arsalan", "malavan", "oes", "hungover", "chicken", "cafés", "mingle", "shawarma", "consumed", "nearest", "gotham", "traded", "ceremonial", "communion", "theoretical", "conjure", "unplayable", "terrorised", "demonstrated", "knockdown", "antigen", "rat", "unjabbed", "quarantine", "rapid", "captive", "obliteration", "disagreement", "divisive", "provocative", "emblematic", "reliably", "kye", "magnetically", "twisting", "manipulating", "sucked", "inconsistent", "snatched", "swimming", "barbecue", "romero", "opting", "insted", "cristian", "discovering", "asados", "ibiza", "liberalism", "reminiscent", "expatriate", "kuwait", "relative", "bottomless", "burden", "disappear", "reveller", "cannier", "flip", "entertained", "baffled", "tipsy", "bbq", "warrant", "bailey", "guilty", "verything", "coal", "trailer", "boycotting", "twickenham", "bowl", "puma", "bonded", "gon", "paroled", "parole", "elli", "fame", "redding", "penitentiary", "boyd", "imprisons", "sexism", "chaperone", "catcall", "hergametoo", "ellie", "mainly", "restrictive", "housed", "pavement", "evacuated", "reuters", "bedding", "mucking", "lucifer", "gwendoline", "revered", "eternal", "gaiman", "comic", "fantastical", "vivienne", "thewlis", "mobility", "tutelage", "hull", "quarrying", "aberdare", "downing", "partygate", "presides", "shagging", "caring", "sue", "gray", "ominous", "saturated", "underloaded", "competitively", "exposing", "stade", "ibrahim", "activated", "moth", "extort", "bot", "repeating", "intervened", "srna", "darijo", "vegan", "boundless", "dart", "whirl", "fro", "mathieu", "swallow", "empathise", "unfamiliar", "ealing", "laughter", "latecomer", "congregational", "dhur", "propelled", "joyously", "recognition", "dazn", "giorgio", "sassuolo", "downtown", "enclave", "barber", "descriptive", "ethnically", "industrial", "pictured", "emailed", "hb", "spelt", "gun", "transport", "robbo", "insatiable", "striving", "excel", "virg", "utilise", "origin", "underpins", "pepijn", "neck", "roh", "gypsy", "berlin", "legitimise", "sportswashing", "burned", "seoul", "adolf", "effigy", "yonsei", "thumama", "animated", "robin", "shorn", "prowess", "reasonable", "joyous", "persie", "pounced", "solly", "lavelle", "spark", "posting", "misguided", "gushing", "perfection", "fruit", "motorbike", "apologist", "lauds", "paradise", "physicality", "amadou", "recommendation", "lille", "dominating", "battleground", "tease", "dijon", "rearrange", "implicit", "interpreted", "jointly", "rambo", "titled", "curating", "vinnie", "swaggering", "browne", "gazza", "poet", "namechecking", "referring", "liable", "rimbaud", "looseners", "icon", "ntil", "bramall", "dean", "dislodge", "permanently", "gea", "pidgeley", "alfreton", "lenny", "pelting", "repel", "sampled", "derbyshire", "darlington", "coalface", "rodney", "incessantly", "ensconced", "ensuing", "nicky", "friendlier", "interfering", "alicia", "aylies", "smet", "verheyden", "playboy", "rau", "fayza", "lamari", "ines", "dispense", "delphine", "bodycams", "pinpoint", "deterrent", "helmet", "monty", "tunic", "chainmail", "python", "toy", "deliberately", "giggling", "guildford", "baby", "emphasise", "holistic", "timid", "demonstrative", "hint", "teen", "disguising", "spotlit", "clearing", "melted", "foe", "arrow", "destroy", "casualty", "martial", "efficient", "indicates", "reinforces", "nursing", "paulo", "tumour", "palliative", "confusion", "são", "colon", "seating", "oversubscribed", "obtained", "finale", "hindered", "crepeau", "maxime", "fray", "enforced", "inaugural", "boothroyd", "dramatically", "aidy", "georgia", "pittsburgh", "rafaela", "pimenta", "torino", "guessed", "curled", "expertly", "dalot", "demeanour", "joão", "cancelo", "grey", "mari", "pan", "quitter", "yvonne", "physiotherapist", "stella", "accor", "lambasted", "outmuscled", "roughed", "outjumped", "outfought", "chastening", "carragher", "slope", "dwell", "ski", "palm", "indoor", "slick", "caput", "frond", "yep", "producer", "stats", "flushed", "viscous", "suburb", "headbutted", "vacated", "salford", "lynne", "worsley", "nacional", "complement", "franchise", "cesc", "malmo", "aired", "spineless", "squabble", "az", "rigour", "negates", "equivalent", "detailing", "flyer", "cheaper", "kate", "hile", "producing", "convey", "thorough", "summarises", "indenture", "kafala", "adamu", "hayatou", "amos", "issa", "almajid", "inherently", "phaedra", "whistleblower", "alleges", "jacques", "anouma", "edict", "tentative", "pronouncement", "unrealistic", "consolidation", "fella", "bedrock", "sabercats", "traditionalist", "zebre", "anayi", "zubimendi", "lokonga", "martín", "sambi", "hotly", "chancellor", "scroll", "caption", "yunus", "cockney", "elle", "purchased", "eclectic", "succinctly", "macpherson", "clinton", "cavernous", "freeman", "fbi", "merton", "aide", "dimitrakopoulos", "michalis", "conscience", "instalment", "mykhailo", "administrator", "netted", "bismark", "weighing", "furthest", "balancing", "offend", "perched", "clan", "harper", "tends", "comrade", "purchase", "contacting", "exerted", "science", "throe", "slipping", "err", "precursor", "georgio", "forlorn", "exhaustive", "flower", "consummate", "assumed", "northamptonshire", "hymn", "gloom", "clandestine", "legally", "locked", "parishioner", "oldham", "priest", "communards", "vicarage", "sequinned", "vicar", "chorister", "finedon", "rev", "bereft", "bawling", "architect", "ageing", "gazing", "combative", "ineffectual", "dejectedly", "tired", "hopping", "succumbs", "downfall", "evicted", "heated", "belies", "intimacy", "mahoney", "overdeliver", "underpromise", "restricted", "mixing", "akanji", "agony", "prefix", "zinedine", "zidane", "courted", "dubbed", "iliman", "futebol", "lickspittle", "supremely", "interacting", "boozing", "jailed", "pomaded", "solely", "vain", "pagan", "hmmm", "airless", "anchor", "narrowly", "brain", "berate", "grande", "survived", "hibbert", "detonated", "ariana", "encore", "wheelchair", "floating", "staircase", "brewed", "ranging", "cocktail", "watering", "flow", "glitz", "deck", "acquisition", "underperformance", "threshold", "upper", "ratio", "yield", "redeeming", "percentage", "gilt", "tormarton", "ranald", "vice", "initiate", "tangled", "gloucestershire", "overbearing", "indifferent", "seville", "magine", "irritating", "carriage", "intrusion", "holidaying", "resentful", "liking", "ignorant", "martinez", "tasting", "wow", "meal", "dish", "certificate", "kenyan", "intensive", "ann", "njue", "pelvic", "wanjiru", "lofty", "safer", "touching", "alphonso", "converting", "nifty", "overqualified", "messiah", "heroic", "threading", "lacazette", "alexandre", "frontline", "villarreal", "gerdine", "lindhout", "retribution", "experiential", "marquinhos", "fábio", "finance", "neto", "nicolas", "valuation", "pépé", "mendes", "havertz", "extorting", "extortion", "judicial", "belonging", "kidnapping", "abduction", "tad", "ccp", "extravagant", "retains", "permeating", "confused", "impose", "undue", "restaurateur", "atalanta", "berth", "contribution", "takumi", "turgid", "install", "scratch", "cooling", "mercury", "cramped", "indicate", "descending", "oving", "refutes", "daytime", "ghoddos", "saman", "dogma", "improper", "clerical", "wasted", "federico", "tirelessly", "rugged", "gatti", "elude", "disrespected", "trudeau", "jeopardised", "cancellation", "revolutionary", "populous", "unsuccessful", "unigate", "dirt", "cs", "motor", "milkman", "rottingdean", "briffa", "ernie", "mechanic", "cleaned", "woodingdean", "grime", "cart", "saltdean", "cinch", "prohibitive", "centralised", "briefed", "supportive", "florida", "ma", "luring", "harvard", "gio", "danielle", "rosalind", "disorderly", "an", "injuring", "exercised", "clouded", "grandson", "dynasty", "fiat", "edoardo", "transformation", "celebratory", "dandy", "elcome", "industrialist", "agnellis", "plummeted", "exchanging", "advance", "blown", "luckily", "confined", "preferring", "théo", "adrien", "shifting", "patrol", "remnant", "grain", "escorted", "farmer", "przewodow", "nato", "barred", "mashhad", "fuelling", "backtracked", "reza", "ridden", "cage", "scooter", "amble", "honey", "waft", "heater", "candyfloss", "strolling", "stoically", "harrowing", "adriatic", "airy", "collapsing", "secede", "eruption", "yugoslavia", "traumatised", "buoyancy", "hyperbole", "peacefully", "unifying", "abide", "notched", "hasselbaink", "cambridge", "vardy", "tielemans", "halted", "returnees", "mk", "pérez", "youri", "don", "brushed", "ayoze", "robust", "method", "enforcement", "defuse", "flashpoint", "unwittingly", "bookmaker", "coates", "sportsbet", "paddy", "aussie", "betfair", "beneficiary", "fanduel", "denise", "noel", "mooney", "briefcase", "merchandising", "perished", "raced", "westerner", "plunging", "billowing", "zac", "shimmering", "alling", "kilter", "michelin", "cork", "commodity", "clonakilty", "delicious", "barrett", "tank", "trundling", "tin", "breathtaking", "ast", "ivanka", "arabella", "luxor", "kushner", "grandpa", "theodore", "giza", "thanksgiving", "temple", "jared", "tomb", "sphinx", "unfinished", "token", "ensuring", "falkland", "bu", "bathroom", "con", "fontas", "afraid", "cadwaladr", "adorning", "rendered", "lament", "cymru", "collar", "retro", "strip", "backlit", "swirly", "badge", "cooler", "conspire", "badly", "winged", "dominik", "collide", "livakovic", "spine", "disposition", "instability", "disasi", "ravaged", "unfamiliarity", "flee", "paradoxically", "galeotti", "untenable", "undermining", "scaling", "disastrously", "argues", "rendering", "mortality", "heralded", "billboard", "temperamental", "grâce", "proudly", "saga", "conquest", "pasture", "steely", "laying", "scuppering", "prophecy", "glut", "doom", "tasted", "somerset", "bearsden", "interval", "qatarantigay", "respectful", "shockingly", "consular", "fightback", "thriller", "indomitable", "pavlovic", "sergej", "conceding", "mitrovic", "aleksandar", "strahinja", "indignation", "marca", "rozas", "vanguardia", "federal", "positioning", "handsomely", "misfired", "tinkered", "logic", "sébastien", "glossy", "priscilla", "pile", "filtering", "momentarily", "outlook", "chemotherapy", "coursed", "matt", "morten", "flogging", "joey", "mirrored", "artwash", "ula", "voiced", "maraya", "pursued", "transforming", "shea", "fortuitous", "wedged", "simón", "rodri", "niclas", "blasting", "forehead", "tooth", "hospitalised", "flu", "disciplinary", "surrender", "ceku", "hajrulla", "colloquially", "mauritius", "anoud", "comoros", "zaporizhzhia", "bombardment", "shelling", "sergiy", "southeastern", "mcdiarmid", "furious", "perth", "applaud", "banknote", "magical", "revaluing", "formality", "rabat", "guitar", "boulevard", "journeyman", "montmartre", "appreciated", "busker", "peering", "amsterdam", "translated", "hunter", "copy", "futurism", "julia", "jonas", "funny", "translator", "tellier", "metoo", "frog", "heale", "campus", "excoriation", "literature", "adriana", "frankly", "abuja", "confederation", "react", "abiola", "stormed", "augustine", "sealed", "eguavoen", "disperse", "moshood", "plotter", "woodward", "vinai", "kluivert", "recount", "batistuta", "resistant", "venkatesham", "emotionally", "racquet", "khelaifi", "hgh", "syringe", "mahogany", "drawer", "cartridge", "insulin", "calle", "marker", "rosario", "commanding", "diabetic", "rectangular", "italia", "hormone", "keown", "omonia", "cyprus", "fiery", "stiffest", "telegram", "monitor", "hateful", "transphobic", "reddit", "detected", "appealed", "improperly", "systematic", "grimacing", "graeme", "wood", "lorraine", "drag", "tumaco", "ecuadorean", "constituent", "absentia", "douglas", "basra", "chapman", "yamagata", "cklamovski", "montedio", "yosuke", "instant", "ideguchi", "yokohama", "tomoki", "egla", "yonhap", "portable", "mingled", "cuban", "bipartisan", "cigar", "ambled", "educational", "mar", "politico", "sushi", "waterside", "stuffy", "ritzy", "illuminated", "rocked", "confessed", "deputy", "parliamentary", "intervening", "repentant", "substantial", "ringleader", "truthful", "westwood", "yule", "dravid", "barrel", "pun", "bangalore", "scraped", "orient", "reboot", "mouthful", "lager", "leyton", "skinner", "anthology", "sleigh", "baddiel", "rahul", "hanged", "tortured", "headhunted", "blindfolded", "deprivation", "hose", "credible", "lynn", "stevenage", "widdringtons", "gunning", "sandwiched", "linnet", "altogether", "dancer", "highflier", "bordeaux", "practically", "avarice", "snigger", "surpass", "objectionable", "hoiking", "dull", "shitshow", "quietest", "iv", "doomed", "mickelson", "enterprise", "drearily", "heckler", "gaudy", "hoarding", "abomination", "restive", "rebellious", "funeral", "residential", "divandarreh", "focal", "gunfire", "helicopter", "unrest", "offensive", "leila", "charm", "foolish", "minnow", "chronicled", "scrabble", "rating", "absurdity", "intimate", "beverly", "hobnobbed", "dollar", "ylian", "nestled", "sodden", "cleft", "engulf", "telephoto", "pane", "contour", "steadicam", "murmurous", "smartphone", "invisible", "armpit", "sinuous", "thunder", "raindrop", "sliding", "roared", "consigned", "troublesome", "itvx", "fumed", "crashing", "scorned", "thankfully", "hath", "facade", "carolyn", "dame", "pours", "deserted", "owards", "straining", "twenty", "scampered", "screeched", "andes", "aboard", "jubilation", "wout", "endlessly", "futile", "withstand", "summed", "virtuoso", "weghorst", "fitter", "ferran", "torres", "twitch", "unwatchable", "vivian", "richards", "antigua", "clashing", "viewing", "tracker", "saving", "pension", "ost", "reluctance", "ftse", "dump", "handy", "waterproof", "pearse", "entice", "crewe", "drinker", "concurred", "strop", "percent", "bola", "evidently", "rebuked", "rail", "tuning", "demonising", "mail", "comparing", "accuse", "maven", "amplified", "heiress", "ruction", "reissuing", "ramping", "nicola", "enninful", "kaoru", "hovering", "cashing", "banished", "encapsulated", "avoiding", "extradite", "hammam", "privy", "palpable", "outstretched", "exhaled", "pointing", "clipping", "letting", "branding", "strengthening", "carmaker", "ea", "gemforex", "maserati", "ltd", "db", "discipline", "eradicate", "uncommon", "supervision", "abyss", "dentist", "irishman", "sprang", "northeast", "pithy", "shrank", "niall", "hoof", "limitless", "elon", "advertiser", "rebuild", "morale", "taskmaster", "stakhanovite", "warhorse", "hype", "patsy", "persona", "punching", "allied", "durable", "antic", "foghorn", "derek", "potent", "yson", "wbc", "batman", "batshit", "cline", "unfiltered", "malaga", "andalucia", "pellegrini", "ought", "tweet", "commemorated", "angrily", "refer", "indignant", "desmond", "frontrunner", "dermot", "internationalist", "bankier", "shareholder", "albeit", "meghan", "hilary", "undertaken", "plagiarism", "navy", "snipped", "commenting", "intervention", "tackled", "fended", "senedd", "joan", "disapproved", "beforehand", "encouragement", "ianni", "contacted", "toni", "remuneration", "weekly", "compulsory", "warns", "missional", "tool", "spitting", "bullied", "delusional", "diatribe", "freckle", "blanked", "outspoken", "endorse", "chevron", "watchdog", "nutrition", "herbal", "massager", "jérôme", "andre", "trueshan", "collared", "landing", "handicap", "goodwood", "stradivarius", "stayer", "shaqab", "sandown", "chester", "waterfront", "showtime", "expressive", "almirón", "imitate", "goalmouth", "bewitching", "reflects", "pater", "belittled", "risking", "spends", "contrition", "disbelief", "warton", "unidentified", "respond", "fahad", "pilot", "arise", "nonetheless", "lancashire", "puskás", "di", "cruyff", "intellectually", "relativity", "ferenc", "denis", "physicist", "geometry", "pythagoras", "thrives", "ronaldos", "fifty", "johan", "artistry", "sorcerer", "stefano", "theatrical", "utopian", "stupid", "nrl", "relevance", "afl", "farted", "wan", "dominates", "adjective", "loyal", "perennially", "liver", "obbes", "fernanda", "pancreatic", "anp", "consumer", "wonderland", "chip", "transforms", "boggled", "spud", "fisherman", "repositioning", "courtesy", "quid", "fish", "christ", "tuber", "kerfuffle", "popping", "retailed", "punchy", "glastonbury", "importing", "apply", "spider", "birthplace", "mist", "picturesque", "nestling", "eildon", "relented", "streaked", "storied", "malaz", "obligation", "marsa", "complied", "kempinski", "recruitment", "conducted", "apologising", "cameroonian", "opinión", "pursues", "altercation", "confrontation", "angered", "riles", "samuel", "amanda", "revolutionise", "orthopaedic", "scribepro", "sfa", "consultant", "mackay", "legendary", "surgeon", "reactionary", "criminalised", "idiot", "zealot", "opposed", "religious", "crimea", "yanai", "yemen", "tadashi", "uniqlo", "boycottuniqlo", "necessity", "retailing", "annexed", "commerce", "retailer", "denim", "megan", "maenad", "atwood", "sasha", "cobbled", "bee", "hawkins", "cavalli", "clattered", "fern", "choo", "alessandra", "inglaterra", "zoo", "separated", "argentinos", "queueing", "wetter", "colder", "sergio", "nerve", "fingernail", "schuchinsky", "shredded", "onwards", "fib", "teamwork", "bonding", "villager", "liar", "anymore", "deception", "lurks", "ganging", "discontent", "wuhan", "iphone", "clause", "affordable", "proving", "wheel", "arrod", "plate", "battered", "fi", "garvey", "emily", "glover", "jane", "marr", "misgiving", "jon", "exodus", "radioland", "occupying", "imbalance", "fled", "straying", "watt", "edgy", "iphones", "ouch", "distributor", "punk", "establishment", "cheeky", "brewer", "ipa", "invest", "turki", "consortium", "sector", "distress", "paramedic", "stretcher", "entertaining", "acute", "heartbroken", "insurance", "location", "anton", "flexible", "description", "terrifying", "evacuation", "tangle", "eveliina", "fearns", "clutched", "amy", "deceiving", "objected", "hampstead", "stringless", "solid", "pupil", "earshot", "butler", "murray", "legion", "smash", "carson", "shhs", "explosive", "squash", "alisson", "equipped", "paquetá", "seleção", "américa", "atone", "casemiro", "fabinho", "regenerative", "felipe", "captaincy", "compiled", "toward", "bitterness", "cheapest", "hampagne", "preference", "champagne", "imaginary", "motioned", "pretend", "succumb", "regrouped", "fenerbahce", "thwarted", "evergreen", "youtube", "rejoiced", "stem", "gleisi", "hoffmann", "riddance", "polarised", "livestream", "resentment", "inácio", "brewing", "register", "cairn", "alun", "ca", "relied", "cynical", "cure", "comfort", "viewed", "fervent", "arriviste", "behaving", "slap", "scrofula", "ailment", "surf", "profiting", "tout", "seatsnet", "allocated", "maternal", "ancestry", "poked", "surname", "eligibility", "hal", "canteen", "unearthed", "shouted", "simon", "referr", "amallah", "fmrf", "awaited", "selim", "sofiane", "césar", "ria", "cyberwar", "offline", "satellite", "protected", "utility", "rutube", "altered", "hacked", "vhs", "honestly", "outcry", "gallery", "fantasised", "publication", "excrement", "paraded", "ramification", "elgin", "threated", "pelted", "outrageous", "diver", "gaming", "active", "entain", "coral", "ladbrokes", "profession", "martyn", "reflex", "lush", "instructive", "concentration", "battering", "rebounding", "margetson", "hammering", "unleash", "rib", "alonso", "marcos", "deduction", "prevention", "phased", "acquired", "ignorance", "socrates", "regaled", "paolo", "falcao", "strode", "jairzinho", "tardelli", "esplendent", "rivellino", "classic", "zico", "gerd", "rossi", "muller", "christensen", "andreas", "nathan", "kamikaze", "sergei", "explode", "palkin", "peaceful", "contravention", "seasoning", "pester", "handbag", "bucket", "sneak", "worn", "confiscated", "benetton", "inexorable", "earmark", "osprey", "gwent", "mythic", "flopped", "ninth", "leftover", "treviso", "historian", "reluctantly", "restarted", "lage", "rowed", "jeff", "lopetugui", "palestinian", "nevertheless", "ordinarily", "accord", "charter", "bahrain", "kilburn", "milian", "belongs", "mortar", "aheem", "paige", "text", "dialled", "cooking", "unanswered", "nancy", "ivo", "raaij", "holdco", "quay", "beauchamps", "recreation", "topco", "hannes", "rogerson", "roefs", "larence", "gratefully", "toll", "brent", "biden", "accepts", "exceptionally", "crude", "mrna", "debtor", "cell", "refinery", "pending", "switching", "zaha", "iñaki", "bilbao", "wilfried", "tariq", "lamptey", "dare", "residing", "eating", "injustice", "originates", "deteriorated", "racking", "fleet", "spat", "unsafe", "operates", "licence", "digital", "core", "müller", "gnabry", "reliant", "goretzka", "leroy", "leon", "joshua", "counterpressing", "serge", "mode", "positional", "sané", "neuer", "kimmich", "jupp", "methodical", "heynckes", "loaning", "lisher", "usmanov", "footstep", "automobile", "glaswegian", "oban", "matthew", "rookie", "simone", "ramsay", "norman", "liberation", "uighur", "province", "roundabout", "gmt", "galeries", "lafayette", "rasmus", "katara", "homegrown", "hbj", "dc", "hedge", "embodiment", "prospective", "washington", "strikingly", "statesman", "scion", "quintessential", "princeling", "immensely", "gallacher", "natalie", "pinkham", "literally", "rochelle", "kaftan", "wavy", "steaming", "marvin", "kirsty", "gaggle", "weird", "blingiest", "hume", "leaning", "sunglass", "completion", "documentation", "alliance", "avoidance", "sulaiti", "formed", "disclosed", "inception", "financed", "vaccinating", "vaccinated", "kremlin", "sovereign", "jab", "exported", "willow", "preview", "master", "magnificent", "crafted", "wg", "combine", "conqueror", "sains", "colossal", "boreham", "stebbing", "hanningfield", "manipulated", "dotcom", "lon", "initiated", "elonmusk", "tweeter", "fraudster", "subscriber", "kim", "megaupload", "unwise", "defunct", "advíncula", "goalline", "jigged", "clowned", "gallese", "unnerve", "cannoned", "forecasting", "referred", "mystery", "ingratitude", "peevish", "epitomises", "honesty", "maldives", "daley", "commonwealth", "slapping", "mêlée", "estadio", "facundo", "infuritated", "infuriated", "pedrera", "fictional", "amie", "daddy", "richmond", "swagger", "sudeikis", "titular", "cocky", "jelmood", "trader", "habib", "advisory", "sri", "lanka", "ounahi", "howling", "azzedine", "bunga", "gorgeous", "separately", "singaporean", "gg", "kerridge", "bartender", "boutique", "discounted", "ofcom", "tom", "waitress", "waiter", "barbra", "aggression", "imprisonment", "blind", "streisand", "fcdo", "paranoia", "oyster", "carpet", "dune", "slave", "overcautious", "intercontinental", "glittering", "refuge", "wooden", "princely", "splendid", "lampshade", "abaya", "staple", "offshoot", "unsettling", "extract", "occupies", "hushed", "spender", "boomtown", "thobes", "headboard", "membership", "compact", "rooftop", "regis", "thrum", "populate", "cecconi", "emphysema", "richer", "lure", "tyrannical", "entitles", "lutyens", "predates", "chattering", "mania", "mint", "toiletry", "sq", "magnet", "hiccup", "squat", "manic", "cowshed", "heave", "creatives", "cute", "hulking", "aesthetic", "burrata", "resplendent", "unfailingly", "neds", "literal", "cacio", "hefty", "logical", "suitably", "howled", "flit", "cloudless", "slather", "desolate", "gasped", "edifice", "impersonate", "impenetrable", "kink", "generic", "password", "artsy", "outpost", "overlooking", "jewel", "waldorf", "atrium", "ambitious", "palatial", "pedestrian", "ft", "introductory", "stylish", "dishevelled", "gaddafi", "sway", "swelter", "teething", "vvip", "aviator", "amusement", "hallway", "execution", "overwrought", "neighbourhood", "custodial", "unnerved", "revoked", "consume", "sin", "peddled", "scrawled", "lucia", "malaysia", "kenya", "interconnected", "dilapidated", "micro", "fobbed", "sharia", "squeaked", "istanbul", "caution", "mozambique", "raided", "admittedly", "visible", "lothario", "birthday", "acted", "montego", "cornflakes", "twentysomething", "amenity", "sinister", "backpacker", "hugh", "risky", "forebear", "crummiest", "patter", "dormant", "scary", "deducing", "gender", "debauchery", "grindr", "collaborative", "forum", "persists", "korzun", "screening", "whisked", "botswana", "definite", "confess", "decriminalised", "perplexingly", "concierge", "honeymoon", "jaike", "boldly", "spectre", "medallist", "weed", "frowned", "angola", "jamaican", "taxi", "enquired", "homosexual", "rum", "inequality", "ostensibly", "leveille", "tainted", "braid", "kin", "ingenuously", "sheikha", "trafficked", "chassis", "tyre", "copper", "tobacco", "reopen", "inland", "shore", "renovated", "renovation", "drilling", "holster", "sudanese", "immaculate", "waver", "abolished", "chunky", "hiss", "peel", "mia", "cloth", "dot", "accelerates", "cleaner", "mesmeric", "strung", "shisha", "saucer", "cinematic", "architecture", "hood", "radwani", "tulun", "implication", "sugar", "paving", "perch", "ware", "collecting", "perfume", "steepest", "doppelgänger", "machinery", "whack", "restored", "henna", "lamp", "flame", "healthcare", "alleyway", "veil", "thread", "snake", "breathless", "incorporate", "tattoo", "darkening", "im", "glassblower", "lighthouse", "flaring", "lightweight", "allows", "submerged", "basket", "incense", "bulb", "headdress", "interlocking", "echo", "specially", "rug", "ruffle", "ibn", "blankly", "bloom", "belt", "lean", "reconnect", "handwritten", "lemon", "slavery", "deepest", "haze", "glare", "wotsit", "stomach", "electricity", "leisurely", "bumping", "bauble", "shameful", "rite", "lonely", "excursion", "discovery", "timeless", "robo", "emerges", "powder", "flatbread", "moza", "shard", "installation", "falconry", "cease", "courtyard", "flash", "painting", "pink", "discernible", "disc", "imaginative", "funnel", "sings", "pad", "stir", "tarmac", "cairo", "baroda", "battleface", "whatsoever", "alaa", "unwarranted", "clare", "kalash", "indolent", "kyte", "imprisoned", "sousse", "enormity", "simmonds", "massacre", "irrational", "srinagar", "territory", "rod", "unjustified", "undermined", "sloppy", "earthquake", "ambiguous", "heed", "kashmir", "thereby", "frustrating", "incompetent", "casson", "random", "welter", "invalidated", "exists", "steppe", "bealby", "nepalese"]]

# Create one list using many lists
data = list(itertools.chain.from_iterable(text_data))
matrix, vocab_index = generate_co_occurrence_matrix(data)


data_matrix = pd.DataFrame(matrix, index=vocab_index,
                             columns=vocab_index)
print(data_matrix)

              scripted  improvement  buggy  linked  sublime  arcadia  abroad  \
scripted           0.0          0.0    0.0     0.0      0.0      0.0     0.0   
improvement        0.0          0.0    0.0     0.0      0.0      0.0     0.0   
buggy              0.0          0.0    0.0     0.0      0.0      0.0     0.0   
linked             0.0          0.0    0.0     0.0      0.0      0.0     0.0   
sublime            0.0          0.0    0.0     0.0      0.0      0.0     0.0   
...                ...          ...    ...     ...      ...      ...     ...   
blindfolded        0.0          0.0    0.0     0.0      0.0      0.0     0.0   
acceleration       0.0          0.0    0.0     0.0      0.0      0.0     0.0   
overbearing        0.0          0.0    0.0     0.0      0.0      0.0     0.0   
heckled            0.0          0.0    0.0     0.0      0.0      0.0     0.0   
prepared           0.0          0.0    0.0     0.0      0.0      0.0     0.0   

              bundesliga  clearer  disa